In [ ]:
# @title 1: Instalar librerías necesarias
!pip install -q -U google-generativeai # Instala/Actualiza

# Instalar spaCy y modelo en español
!pip install -q spacy

#@title Selecciona el modelo de spaCy
# @markdown # Elige un modelo de spacy para separar por frases en textos largos
modelo = "es_core_news_lg" #@param ["es_core_news_sm", "es_core_news_md", "es_core_news_lg"]

!pip install -q spacy
!python -m spacy download $modelo

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 568.0/568.0 MB 1.7 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_lg')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [105]:
# @title 2: Importar librerías y configurar API Key
import re
import time
import pandas as pd
import csv # Necesario para quoting en to_csv
import google.generativeai as genai
import json
import os
from google.colab import userdata # Para manejar secrets en Colab

# --- Configuración de la API de Gemini ---
# Nota: Debes guardar tu API key en los 'Secrets' de Colab con el nombre 'GOOGLE_API_KEY'
# Si no usas secrets, descomenta la siguiente línea y pega tu key (menos seguro)
# GOOGLE_API_KEY="TU_API_KEY_AQUI"
try:
    GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
    genai.configure(api_key=GOOGLE_API_KEY)
    print("API Key de Gemini configurada correctamente desde Secrets.")
except userdata.SecretNotFoundError:
    print("Advertencia: Secret 'GOOGLE_API_KEY' no encontrado.")
    print("Por favor, añade tu API key a los 'Secrets' de Colab (Panel izquierdo -> Icono de llave).")
    GOOGLE_API_KEY = None
except Exception as e:
    print(f"Ocurrió un error al configurar la API Key: {e}")
    GOOGLE_API_KEY = None

# Configuración del modelo Gemini a usar
# Puedes cambiar a "gemini-1.5-flash" si prefieres velocidad/menor coste
model_name = "gemini-2.0-flash" # @param ["gemini-2.0-flash-lite", "gemini-2.0-flash", "gemini-1.5-flash", "gemini-1.5-pro-latest", "gemini-2.5-pro-preview-03-25"]

if GOOGLE_API_KEY:
    model = genai.GenerativeModel(model_name)
    print(f"Modelo Gemini '{model_name}' inicializado.")
else:
    model = None
    print("El modelo Gemini no pudo ser inicializado por falta de API Key.")

API Key de Gemini configurada correctamente desde Secrets.
Modelo Gemini 'gemini-2.0-flash' inicializado.


In [106]:
# @title 3: Definir los Textos a Analizar

# Cambiamos el nombre de la variable para mayor claridad

textos_para_analizar = [
    # Textos cortos:
    "Me gustaría cortarle el pelo, ya está muy largo",
    "Necesito ir al banco.",
    "Él vino tarde y trajo el vino que prometió.",
    "La copia del informe está lista; el estudiante copia descaradamente.",
    "Bajo la intensa lluvia, bajo al refugio rápidamente.",
    "Vi al hombre con el telescopio.",
    "Compramos pasteles y bebidas frías para la fiesta.",
    "El perro persiguió al gato hasta que se cansó.",
    "El sol brilla fuerte y los niños juegan alegres en el parque.",
    "El libro que me recomendaste ayer tiene una trama muy compleja.",
    "¡Caramba!, no esperaba encontrarte aquí.",
    "Me va a costar un ojo de la cara",
    "pucha vecino, no me quedan más bolsitas",
    "Tía no hay no como que eso pepino nada tortuga y sandía realmente elementales de un pájaro rueda corri vez lo que me dijiste amistades ruines" # Frase agramatical/random
    "Los corazones palpitan y palpitan",
    "Tengo ganas de interactuar con la página",

    # Texto Largo 1: La canción
    """Esta enfermedad de amanecer nostálgico
En la oscuridad de madrugada sórdida
Tanto tiempo andando en piloto automático
Navegando pa' no pisar tierra sólida

Creo que he recordado esta mañana mágica
Tardes de tu mano en ese puerto trágico
Donde tu mirada era la Luna única
En nuestro desierto de absoluta oscuridad

Astronauta dónde anda tu nave de cristal
Donde construimos un hogar galáctico
Acaso surcando el universo práctico
O en otro planeta de la vía láctea

Mi llorar es un asunto sintomático
De mi propia nave que anda a toda máquina
Deshaciéndose entre numerosos cánticos
Disparando a las murallas de la humanidad

Esta enfermedad de amanecer nostálgico
Que no se me pasa ni con la felicidad
Me tiene añorando algún contacto tántrico
Con esa mirada de infinita locura

Creo que he recordado esta mañana trágica
Tardes de tu mano en ese puerto mágico
Donde las palabras eran mero trámite
De nuestros lenguajes fuera de la lógica

Astronauta dónde fue tu nave de cristal
Donde construimos un hogar galáctico
Acaso surcando el universo práctico
O en otro planeta de la vía láctea""",

    # Texto Largo 2: Ejemplo de Noticia Corta
    """El presidente anunció nuevas medidas económicas durante la conferencia de prensa matutina. Según explicó el mandatario, el objetivo principal es contener la inflación galopante que afecta el bolsillo de los ciudadanos. Analistas económicos recibieron las propuestas con escepticismo, señalando la falta de detalles concretos sobre su implementación y financiación. La oposición, por su parte, criticó duramente el paquete, calificándolo de "insuficiente" y "electoralista". Se espera un debate intenso en el Congreso durante las próximas semanas.""",

    # --- AÑADE AQUÍ MÁS TEXTOS LARGOS COMPLETOS COMO STRINGS SEPARADOS POR COMAS ---
    # Ejemplo:
    """Ahí va el capitán Beto por el espacio
Con su nave de fibra hecha en Haedo
Ayer colectivero
Hoy amo entre los amos del aire

Ya lleva quince años en su periplo
Su equipo es tan precario como su destino
Sin embargo un anillo extraño
Ahuyenta sus peligros en el cosmos

Ahí va el capitán Beto por el espacio
La foto de Carlitos sobre el comando
Y un banderín de River Plate
Y la triste estampita de un santo

¿Dónde está el lugar
Al que todos llaman cielo?
Si nadie viene hasta aquí
A cebarme unos amargos
Como en mi viejo umbral

¿Por qué habré venido hasta aquí?
Si no puedo más de soledad
Ya no puedo más de soledad

Su anillo lo inmuniza de los peligros
Pero no lo protege de la tristeza
Surcando la galaxia del hombre
Ahí va el capitán Beto, el errante

¿Dónde habrá una ciudad
En la que alguien silbe un tango?
¿Dónde están, dónde están
Los camiones de basura
Mi vieja y el café?
Si esto sigue así como así
Ni una triste sombra quedará
Ni una triste sombra quedará

Ahí va el capitán Beto por el espacio
Regando los malvones de su cabina
Sin brújula y sin radio
Jamás podrá volver a la tierra

Tardaron muchos años hasta encontrarlo
El anillo de Beto llevaba inscripto
Un signo del alma""",
    """La bruma espesa, eterna, para que olvide dónde
me ha arrojado la mar en su ola de salmuera.
La tierra a la que vine no tiene primavera:
tiene su noche larga que cual madre me esconde.
El viento hace a mi casa su ronda de sollozos
y de alarido, y quiebra, como un cristal, mi grito.
Y en la llanura blanca, de horizonte infinito,
miro morir intensos ocasos dolorosos.
¿A quién podrá llamar la que hasta aquí ha venido
si más lejos que ella sólo fueron los muertos?
¡Tan sólo ellos contemplan un mar callado y yerto
crecer entre sus brazos y los brazos queridos!
Los barcos cuyas velas blanquean en el puerto
vienen de tierras donde no están los que son míos;
y traen frutos pálidos, sin la luz de mis huertos,
sus hombres de ojos claros no conocen mis ríos.
Y la interrogación que sube a mi garganta
al mirarlos pasar, me desciende, vencida:
hablan extrañas lenguas y no la conmovida
lengua que en tierras de oro mi vieja madre canta.
Miro bajar la nieve como el polvo en la huesa;
miro crecer la niebla como el agonizante,
y por no enloquecer no encuentro los instantes,
porque la "noche larga" ahora tan solo empieza.
Miro el llano extasiado y recojo su duelo,
que vine para ver los paisajes mortales.
La nieve es el semblante que asoma a mis cristales;
¡siempre será su altura bajando de los cielos!
Siempre ella, silenciosa, como la gran mirada
de Dios sobre mí; siempre su azahar sobre mi casa;
siempre, como el destino que ni mengua ni pasa,
descenderá a cubrirme, terrible y extasiada.""",

]

print(f"Se analizarán {len(textos_para_analizar)} textos.")

# Añadimos aquí la importación de spaCy y la carga del modelo
# para asegurarnos de que esté listo antes de la Celda 4.
# (Si prefieres mantener las importaciones agrupadas, asegúrate
#  de que spaCy esté importado antes de usarlo en Celda 4)
try:
    import spacy
    # *** ASEGÚRATE DE QUE ESTE NOMBRE COINCIDA CON EL MODELO QUE INSTALASTE ***
    # (Ej: "es_core_news_lg", "es_core_news_md", "es_dep_news_trf")
    nlp_spacy_loader = spacy.load("es_core_news_lg")
    print(f"Modelo spaCy 'es_core_news_lg' precargado para segmentación.")
except ImportError:
    print("Advertencia: spaCy no está instalado. La segmentación fallará.")
    nlp_spacy_loader = None
except OSError:
    print("Error: Modelo spaCy 'es_core_news_lg' no encontrado. ¿Ejecutaste la celda de instalación?")
    print("Verifica el nombre del modelo ('es_core_news_sm', 'md', 'lg', 'trf') y que esté descargado.")
    nlp_spacy_loader = None

Se analizarán 19 textos.
Modelo spaCy 'es_core_news_lg' precargado para segmentación.


In [107]:
# @title 4: Segmentar Textos (spaCy y Saltos de Línea si spaCy Falla)

# --- Parámetros ---
#@markdown ### Opciones de Segmentación
#@markdown Activar para intentar dividir por saltos de línea **SÓLO SI** spaCy no logra dividir el texto en múltiples frases (útil para poesía/canciones sin puntuación).
intentar_segmentacion_por_lineas_si_falla_spacy = True #@param {type:"boolean"}

import textwrap # Para mostrar los textos largos de forma más legible

print("Iniciando la segmentación de textos...\n")

# Verificar que el modelo spaCy ('nlp_spacy_loader') esté cargado
if nlp_spacy_loader:
    resultados_segmentacion_spacy = {} # Diccionario para frases de spaCy
    resultados_segmentacion_lineas = {} # Diccionario para líneas (se llenará condicionalmente)

    # Iterar sobre cada texto en la lista textos_para_analizar
    for i, texto_completo in enumerate(textos_para_analizar):
        print(f"--- Procesando Texto {i+1} ---")
        print("Texto Original (primeros 200 caracteres):")
        print(textwrap.fill(texto_completo[:200], width=80))
        print("-" * 50)

        # --- 1. Segmentación con spaCy (Siempre se intenta) ---
        print("Frases Detectadas por spaCy (Gramatical):")
        doc = nlp_spacy_loader(texto_completo)
        frases_detectadas_spacy = []
        # Usamos una lista temporal para poder contarla antes de imprimir
        temp_frases_spacy = []
        for sent in doc.sents:
            frase_limpia = sent.text.strip()
            if frase_limpia:
                temp_frases_spacy.append(frase_limpia)

        # Ahora imprimimos las frases encontradas
        if temp_frases_spacy:
            for j, frase in enumerate(temp_frases_spacy):
                 print(f"  SpaCy Frase {j+1}: {frase}")
            frases_detectadas_spacy = temp_frases_spacy # Asignar a la variable final
        else:
            print("  (spaCy no detectó frases)")

        num_frases_spacy = len(frases_detectadas_spacy)
        print(f"Total frases spaCy: {num_frases_spacy}")
        resultados_segmentacion_spacy[f"Texto_{i+1}"] = frases_detectadas_spacy
        print("-" * 50)

        # --- 2. Segmentación por Saltos de Línea (Condicional) ---
        # Comprobar si la opción está activada Y si spaCy encontró 0 o 1 frase
        realizar_segmentacion_lineas = False # Bandera para saber si se hizo
        if intentar_segmentacion_por_lineas_si_falla_spacy and num_frases_spacy <= 1:
            print("Intentando segmentación por saltos de línea (spaCy encontró <= 1 frase):")
            # Dividir el texto por el carácter de nueva línea
            lineas_raw = texto_completo.split('\n')
            lineas_detectadas = []
            for j, linea in enumerate(lineas_raw):
                linea_limpia = linea.strip() # Limpiar espacios
                if linea_limpia: # Solo mostrar/guardar si no está vacía
                  print(f"  Línea {j+1}: {linea_limpia}")
                  lineas_detectadas.append(linea_limpia)

            if lineas_detectadas:
                print(f"Total líneas detectadas: {len(lineas_detectadas)}")
                # Guardar en el diccionario SOLO si se realizó la segmentación y se encontraron líneas
                resultados_segmentacion_lineas[f"Texto_{i+1}"] = lineas_detectadas
                realizar_segmentacion_lineas = True
            else:
                print("  (No se encontraron líneas no vacías)")

        elif intentar_segmentacion_por_lineas_si_falla_spacy and num_frases_spacy > 1:
            print("Segmentación por saltos de línea OMITIDA (spaCy ya dividió el texto en múltiples frases).")

        #else: # Si la casilla estaba desmarcada
            # Opcional: Mensaje si la opción está desactivada
            # if not intentar_segmentacion_por_lineas_si_falla_spacy:
            #    print("Segmentación por saltos de línea desactivada por el usuario.")

        # Imprimir separador de líneas solo si se hizo algo en la sección 2
        if realizar_segmentacion_lineas or (intentar_segmentacion_por_lineas_si_falla_spacy and num_frases_spacy > 1):
             print("-" * 50)


        print("\n" + "="*70 + "\n") # Separador visual entre textos

else:
    print("¡ERROR! No se pudo encontrar el modelo spaCy ('nlp_spacy_loader').")
    # ... (resto del manejo de error igual) ...
    resultados_segmentacion_spacy = None
    resultados_segmentacion_lineas = None

print("Segmentación y visualización completadas.")

# Opcional: Mostrar los diccionarios resultantes
# import pprint
#if resultados_segmentacion_spacy:
#  print("\nFrases detectadas por spaCy:")
#  pprint.pprint(resultados_segmentacion_spacy)
# Nota: resultados_segmentacion_lineas podría estar vacío o no tener todas
# las claves si la opción estaba desactivada durante la ejecución.
#if resultados_segmentacion_lineas:
#  print("\nLíneas detectadas por salto de línea (si estaba activado):")
#  pprint.pprint(resultados_segmentacion_lineas)

Iniciando la segmentación de textos...

--- Procesando Texto 1 ---
Texto Original (primeros 200 caracteres):
Me gustaría cortarle el pelo, ya está muy largo
--------------------------------------------------
Frases Detectadas por spaCy (Gramatical):
  SpaCy Frase 1: Me gustaría cortarle el pelo, ya está muy largo
Total frases spaCy: 1
--------------------------------------------------
Intentando segmentación por saltos de línea (spaCy encontró <= 1 frase):
  Línea 1: Me gustaría cortarle el pelo, ya está muy largo
Total líneas detectadas: 1
--------------------------------------------------


--- Procesando Texto 2 ---
Texto Original (primeros 200 caracteres):
Necesito ir al banco.
--------------------------------------------------
Frases Detectadas por spaCy (Gramatical):
  SpaCy Frase 1: Necesito ir al banco.
Total frases spaCy: 1
--------------------------------------------------
Intentando segmentación por saltos de línea (spaCy encontró <= 1 frase):
  Línea 1: Necesito ir al banco

In [108]:
# @title 5: Definición de esquemas (Estos definen la estructura de los datos que incluiremos en el dataset)

# --- Definiciones (Schemas, Mapeo por categoría gramatical) ---
schemas = {
    "sustantivo": {"categoria": "sustantivo", "palabra_analizada": None, "lemma": None, "genero": None, "numero": None, "tipo": None, "diminutivo_comun": None, "aumentativo_comun": None, "definicion_contextual": None, "definicion_general": None, "ejemplos_uso": []},
    "adjetivo": {"categoria": "adjetivo", "palabra_analizada": None, "lemma": None, "genero": None, "numero": None, "grado": None, "apocope": None, "modifica_a": None, "definicion_general": None, "ejemplos_uso": []},
    "verbo": {"categoria": "verbo", "palabra_analizada": None, "infinitivo": None, "modo": None, "tiempo": None, "persona": None, "numero": None, "participio": None, "gerundio": None, "transitividad": None, "definicion_general": None, "ejemplos_uso": []},
    "determinante": {"categoria": "determinante", "palabra_analizada": None, "tipo": None, "subtipo": None, "genero": None, "numero": None, "persona": None, "distancia": None, "definicion_funcion": None, "ejemplos_uso": []},
    "pronombre": {"categoria": "pronombre", "palabra_analizada": None, "tipo": None, "persona": None, "genero": None, "numero": None, "caso": None, "tonicidad": None, "referente_aproximado": None, "definicion_funcion": None, "ejemplos_uso": []},
    "adverbio": {"categoria": "adverbio", "palabra_analizada": None, "lemma": None, "tipo": None, "grado": None, "modifica_a": None, "definicion_general": None, "ejemplos_uso": []},
    "preposicion": {"categoria": "preposición", "palabra_analizada": None, "tipo": None, "usos_comunes": [], "definicion_funcion": "Introduce un complemento indicando una relación específica.", "ejemplos_uso": []},
    "conjuncion": {"categoria": "conjunción", "palabra_analizada": None, "tipo": None, "subtipo": None, "definicion_funcion": "Une palabras, sintagmas u oraciones.", "ejemplos_uso": []},
    "interjeccion": {"categoria": "interjección", "palabra_analizada": None, "tipo": None, "subtipo": None, "emocion_tipica": None, "definicion_funcion": "Expresa emociones súbitas o llama la atención.", "ejemplos_uso": []},
    "puntuacion": {"categoria": "puntuacion", "palabra_analizada": None, "tipo": None, "funcion": None},
    "otro": {"categoria": "otro", "palabra_analizada": None, "descripcion": "Categoría no identificada o palabra desconocida."}
}

category_map = {
    "sustantivo": "sustantivo", "noun": "sustantivo", "adjetivo": "adjetivo", "adj": "adjetivo",
    "verbo": "verbo", "verb": "verbo", "aux": "verbo", "determinante": "determinante", "det": "determinante",
    "artículo": "determinante", "pronombre": "pronombre", "pron": "pronombre", "adverbio": "adverbio", "adv": "adverbio",
    "preposición": "preposicion", "adp": "preposicion", "conjunción": "conjuncion", "conj": "conjuncion",
    "cconj": "conjuncion", "sconj": "conjuncion", "interjección": "interjeccion", "intj": "interjeccion",
    "puntuación": "puntuacion", "punct": "puntuacion", "numeral": "determinante", "num": "determinante",
    "partícula": "otro", "part": "otro", "símbolo": "otro", "sym": "otro", "contracción": "otro",
    "otro": "otro", "x": "otro"
}

# --- Fin Definiciones ---

In [109]:
# @title 6: Generar Análisis CoT Detallado por Segmento

import re
import time
import json # Necesario para guardar potencialmente errores
from google.api_core import exceptions

# --- Verificaciones Previas ---
if 'model' not in globals() or model is None:
    raise ValueError("Modelo Gemini ('model') no inicializado. Ejecuta la Celda 2.")
if 'textos_para_analizar' not in globals():
    raise NameError("La lista 'textos_para_analizar' no se encontró. Ejecuta la Celda 3.")
if 'resultados_segmentacion_spacy' not in globals():
     raise NameError("Resultados de segmentación de spaCy no encontrados. Ejecuta la Celda 4.")
if 'schemas' not in globals():
    raise NameError("Los 'schemas' no se encontraron. Ejecuta la Celda 5.")
# nlp_spacy_loader sigue siendo útil para referencia futura o validación, aunque no lo usemos activamente aquí
if 'nlp_spacy_loader' not in globals():
    print("⚠️ Advertencia: 'nlp_spacy_loader' no encontrado (Celda 3).")
    nlp_spacy_loader = None

# Obtener la lista de categorías válidas de los schemas
lista_categorias_validas = list(schemas.keys())
categorias_string = ", ".join(lista_categorias_validas)
print(f"Categorías gramaticales a usar (de schemas): {categorias_string}")

print("\n--- Iniciando Generación de Análisis CoT Detallado por Segmento ---")

# Diccionario para almacenar los análisis CoT detallados
resultados_analisis_cot = {}

# Parámetros
TEMPERATURA_COT_DETALLADO = 0.5 # Algo de creatividad puede ser útil para la reflexión

# Iterar sobre cada texto original
total_textos = len(textos_para_analizar)
for i, texto_original in enumerate(textos_para_analizar):
    id_texto = f"Texto_{i+1}"
    print(f"\n L ({i+1}/{total_textos}) Procesando CoT Detallado para: {id_texto}")
    print(f"Texto original (fragmento): {texto_original[:100]}...")

    resultados_analisis_cot[id_texto] = {
        "texto_original": texto_original,
        "tipo_segmentacion_usada": None,
        "analisis_por_segmento": [] # Lista para guardar el CoT de cada segmento
    }

    # --- Selección de Segmentos (misma lógica que antes) ---
    lista_segmentos_a_analizar = []
    tipo_segmentacion = "Desconocido"
    if id_texto in resultados_segmentacion_spacy and len(resultados_segmentacion_spacy.get(id_texto, [])) > 1:
        lista_segmentos_a_analizar = resultados_segmentacion_spacy[id_texto]
        tipo_segmentacion = "Frases (spaCy)"
    elif id_texto in resultados_segmentacion_lineas and resultados_segmentacion_lineas.get(id_texto):
         lista_segmentos_a_analizar = resultados_segmentacion_lineas[id_texto]
         tipo_segmentacion = "Líneas (Fallback)"
    elif id_texto in resultados_segmentacion_spacy and resultados_segmentacion_spacy.get(id_texto):
         lista_segmentos_a_analizar = resultados_segmentacion_spacy[id_texto]
         tipo_segmentacion = "Frase única (spaCy)"
    else:
        print(f"⚠️ Advertencia: No se encontraron segmentos válidos para {id_texto}. Saltando análisis CoT para este texto.")
        tipo_segmentacion = "Ninguna Válida"

    print(f"Segmentación seleccionada: {tipo_segmentacion} ({len(lista_segmentos_a_analizar)} segmentos)")
    resultados_analisis_cot[id_texto]["tipo_segmentacion_usada"] = tipo_segmentacion

    if not lista_segmentos_a_analizar:
        continue # Saltar al siguiente texto si no hay segmentos

    # --- Iterar por cada Segmento para generar el CoT Detallado ---
    analisis_segmentos_lista = []
    total_segmentos = len(lista_segmentos_a_analizar)
    for j, texto_segmento in enumerate(lista_segmentos_a_analizar):
        print(f"  - Segmento {j+1}/{total_segmentos}: Generando CoT Detallado...")
        segmento_resultado = {
            "segmento_texto": texto_segmento,
            "analisis_cot_completo": None,
            "error": None
        }

        # --- Construir y Ejecutar el Prompt CoT Detallado ---
        try:
            # Adaptamos tu prompt detallado
            prompt_cot_detallado = f"""
**Tarea Principal:** Actúa como un lingüista experto y realiza un análisis crítico, detallado y estructurado del siguiente segmento de texto en español. Cuestiona las primeras impresiones y justifica tus conclusiones basándote en principios lingüísticos.

**Segmento Específico a Analizar:**
---
'{texto_segmento}'
---

**Contexto General (Texto Original Completo - Úsalo para desambiguar):**
---
{texto_original}
---

**Proceso de Análisis Lingüístico (Chain-of-Thought - Sigue este orden ESTRICTAMENTE):**

**1. Reflexión Inicial y Contextualización:** (Empieza con una visión general y crítica)
    *   **1.1. Contexto Inferido (Obligatorio):** Infiere y justifica rigurosamente el tipo de texto más probable (poético/lírico, narrativo, periodístico, conversacional, técnico, etc.), el registro (formal, informal, especializado, arcaico...), y la posible intención pragmática dominante del *segmento* dentro del texto completo (expresar emoción, informar, describir, persuadir, etc.). Identifica posibles **indicios clave** (léxico, estructura inicial) que sugieran este contexto. Si aplica (ej. poético), menciona posibles recursos estilísticos generales que podrían esperarse o intuirse.
    *   **1.2. Ambigüedades Potenciales Preliminares:** Basándote SÓLO en la lectura inicial del segmento y el contexto inferido, ¿existen palabras o estructuras que *a primera vista* parezcan potencialmente ambiguas (léxica, sintáctica, referencialmente)? Enuméralas brevemente como hipótesis a verificar más adelante. Si ninguna parece obvia inicialmente, indica "Ninguna ambigüedad evidente a primera vista, sujeto a análisis posterior."
    *   **1.3. Significado e Interpretación Hipotética:** Basado en 1.1 y 1.2, formula una hipótesis inicial sobre el significado literal principal y cualquier posible significado implícito o connotativo del segmento. ¿Qué idea o sentimiento parece transmitir preliminarmente?

**2. Estructura Básica del Segmento:**
    *   **2.1. Tipo de Estructura:** Clasifica la estructura general del segmento (ej: Oración simple, Oración compuesta [indicar tipo: coordinada/subordinada], Frase nominal, Verso, Interjección aislada, etc.). Justifica brevemente tu clasificación basándote en la presencia/ausencia de verbos conjugados, nexos, etc., y considerando el contexto (Paso 1).
    *   **2.2. Componentes Sintácticos/Ideacionales Clave:** Identifica los constituyentes principales según la estructura anterior (ej: Sujeto [explícito/omitido/impersonal], Núcleo Verbal, Complementos esenciales [CD, CI, Atributo, etc.], o bien, el núcleo de la frase nominal, la idea central del verso, etc.).

**3. Análisis Léxico-Gramatical Contextualizado:** (AHORA, y basándote en la reflexión (Paso 1) y la estructura (Paso 2), analiza cada palabra)
    (Formato: Palabra | Ambigüedad Resuelta/Persistente [Sí/No/Nota breve de resolución] | Categoría Gramatical Final Asignada | Justificación breve *solo si la asignación fue compleja o dependió fuertemente del contexto*)
    *   'Palabra1': [Ambig. Resuelta] | [Cat. Gram. Final] | [Justif. si aplica]
    *   'Palabra2': [Ambig. Resuelta] | [Cat. Gram. Final] | [Justif. si aplica]
    (Continúa para **todas** las palabras y signos de puntuación del segmento. **Usa OBLIGATORIAMENTE una de las siguientes categorías gramaticales para [Cat. Gram. Final]: {categorias_string}**. Refleja aquí cómo el contexto resolvió (o no) las ambigüedades de 1.2).

**4. Revisión Crítica y Resolución de Conflictos:** (Compara y contrasta los hallazgos)
    *   **4.1. Consistencia Interna:** ¿Son coherentes los hallazgos de los Pasos 1, 2 y 3? ¿El análisis léxico-gramatical detallado (Paso 3) confirma o refuta las hipótesis iniciales sobre contexto, ambigüedad y significado (Paso 1)? ¿La estructura identificada (Paso 2) soporta la interpretación? Señala explícitamente cualquier tensión o confirmación.
    *   **4.2. Resolución Definitiva de Ambigüedades:** Revisa las ambigüedades potenciales (1.2) y las resoluciones (Paso 3). ¿Se resolvieron todas satisfactoriamente gracias al contexto y análisis estructural? Si alguna persiste, explica por qué y qué interpretaciones alternativas válidas permite. Si no hubo ambigüedades relevantes, indícalo.
    *   **4.3. Aspectos Lingüísticos Destacables:** ¿Hay algún fenómeno lingüístico particularmente interesante en este segmento (uso inusual de una palabra, estructura sintáctica marcada, figura retórica confirmada, elipsis significativa, etc.) que merezca mención especial tras el análisis completo?

**5. Síntesis Lingüística Final:** (Resume la esencia del análisis)
    *   Resume en 2-3 frases concisas las características lingüísticas clave del segmento (complejidad, claridad/ambigüedad final), su función probable dentro del texto mayor, y cómo su forma y contexto interactúan para crear el significado final interpretado.

**Instrucciones Finales:**
*   Adopta un tono analítico y crítico de lingüista.
*   Sigue el orden y formato de los 5 pasos y subpuntos rigurosamente.
*   Sé exhaustivo donde se requiere detalle (Pasos 1, 3, 4) pero conciso en las justificaciones si son obvias.
*   Utiliza terminología lingüística apropiada y consistente.
*   Enfócate exclusivamente en el análisis del *segmento proporcionado*, usando el texto completo solo como ayuda contextual.
"""
            generation_config_cot = genai.types.GenerationConfig(temperature=TEMPERATURA_COT_DETALLADO)
            safety_settings = safety_settings if 'safety_settings' in globals() else None # Reutilizar config de seguridad si existe

            response_cot = model.generate_content(
                prompt_cot_detallado,
                generation_config=generation_config_cot,
                safety_settings=safety_settings
                )
            cot_detallado_output = response_cot.text.strip()
            segmento_resultado["analisis_cot_completo"] = cot_detallado_output
            print(f"    ✅ CoT Detallado generado.")

        except exceptions.ResourceExhausted as e_rate:
            print(f"  ⛔ ERROR API (Rate Limit) en CoT Detallado para segmento {j + 1}: {e_rate}. Esperando 60s...")
            segmento_resultado["error"] = f"ERROR API (Rate Limit): {e_rate}"
            time.sleep(60)
            # Considerar añadir reintento aquí
        except Exception as e_cot:
            print(f"  ⛔ Error generando CoT Detallado para segmento {j + 1}: {e_cot}")
            try: error_text = response_cot.text # Intentar obtener texto si hubo respuesta parcial
            except: error_text = str(e_cot)
            segmento_resultado["error"] = f"ERROR: {error_text}"

        analisis_segmentos_lista.append(segmento_resultado)

        # Pausa entre llamadas a segmentos (quizás un poco más larga para este prompt más complejo)
        time.sleep(4) # Pausa de 4 segundos (ajusta según necesidad/modelo)

    # Guardar la lista de análisis de segmentos para este texto
    resultados_analisis_cot[id_texto]["analisis_por_segmento"] = analisis_segmentos_lista

print("\n--- Generación de Análisis CoT Detallado Completada ---")

# Opcional: Mostrar un ejemplo del resultado CoT guardado para el primer texto
import pprint
if resultados_analisis_cot:
    primer_id = list(resultados_analisis_cot.keys())[0]
    print(f"\nEjemplo de resultado CoT Detallado para {primer_id} (primer segmento):")
    if resultados_analisis_cot[primer_id]["analisis_por_segmento"]:
        # Imprimir solo el texto del CoT del primer segmento para no saturar
        primer_segmento_info = resultados_analisis_cot[primer_id]["analisis_por_segmento"][0]
        print(f"\n--- Segmento: {primer_segmento_info['segmento_texto']} ---")
        if primer_segmento_info['analisis_cot_completo']:
             print(primer_segmento_info['analisis_cot_completo'])
        else:
             print(f"Error: {primer_segmento_info['error']}")
    else:
        print("(No se analizaron segmentos para este texto)")

else:
    print("\nNo se generaron resultados CoT Detallados.")

Categorías gramaticales a usar (de schemas): sustantivo, adjetivo, verbo, determinante, pronombre, adverbio, preposicion, conjuncion, interjeccion, puntuacion, otro

--- Iniciando Generación de Análisis CoT Detallado por Segmento ---

 L (1/19) Procesando CoT Detallado para: Texto_1
Texto original (fragmento): Me gustaría cortarle el pelo, ya está muy largo...
Segmentación seleccionada: Líneas (Fallback) (1 segmentos)
  - Segmento 1/1: Generando CoT Detallado...
    ✅ CoT Detallado generado.

 L (2/19) Procesando CoT Detallado para: Texto_2
Texto original (fragmento): Necesito ir al banco....
Segmentación seleccionada: Líneas (Fallback) (1 segmentos)
  - Segmento 1/1: Generando CoT Detallado...
    ✅ CoT Detallado generado.

 L (3/19) Procesando CoT Detallado para: Texto_3
Texto original (fragmento): Él vino tarde y trajo el vino que prometió....
Segmentación seleccionada: Líneas (Fallback) (1 segmentos)
  - Segmento 1/1: Generando CoT Detallado...
    ✅ CoT Detallado generado.

 L (4/

In [110]:
# @title 7 (Revisado): Generar JSON Detallado por Palabra (Salida Limpia)

import json
import time
import re
from google.api_core import exceptions
from IPython.display import display, Markdown, clear_output # Para barra de progreso
import ipywidgets as widgets # Para barra de progreso

# --- Verificaciones Previas ---
if 'model' not in globals() or model is None: raise ValueError("Modelo Gemini no inicializado (Celda 2).")
if 'resultados_analisis_cot' not in globals() or not resultados_analisis_cot: raise NameError("Resultados CoT detallado no encontrados (Celda 6).")
if 'schemas' not in globals(): raise NameError("Schemas no encontrados (Celda 5).")

# Preparar schemas como string JSON
try:
    schemas_json_string = json.dumps(schemas, indent=2, ensure_ascii=False)
    # print("✅ Schemas JSON preparados.") # Comentado para limpieza
except Exception as e:
    print(f"⚠️ Advertencia: No se pudo convertir schemas a JSON: {e}"); schemas_json_string = "{...}"

print("\n--- Iniciando Generación de Análisis JSON Detallado por Palabra ---")

# Diccionario para almacenar resultados
resultados_json_final_revisado = {}

# Parámetros
TEMPERATURA_JSON = 0.2
MAX_JSON_RETRIES = 1

# --- Barra de Progreso ---
total_segmentos_global = sum(len(data.get("analisis_por_segmento", [])) for data in resultados_analisis_cot.values())
progress_bar = widgets.FloatProgress(value=0.0, min=0.0, max=1.0, description='Progreso:', bar_style='info', orientation='horizontal')
progress_label = widgets.Label(value="0/0 Segmentos")
display(widgets.VBox([progress_label, progress_bar]))
segmentos_procesados_count = 0
errores_json_count = 0
errores_api_count = 0
# ----------------------

# Iterar sobre los textos
total_textos_procesados = len(resultados_analisis_cot)
for i, (id_texto, data_texto) in enumerate(resultados_analisis_cot.items()):
    # Actualizar etiqueta para indicar texto actual (opcional)
    # progress_label.value = f"Procesando Texto {i+1}/{total_textos_procesados}..."

    texto_original_completo = data_texto["texto_original"]
    analisis_por_segmento_cot = data_texto.get("analisis_por_segmento", [])

    resultados_json_final_revisado[id_texto] = {
        "texto_original": texto_original_completo,
        "tipo_segmentacion_usada": data_texto.get("tipo_segmentacion_usada"),
        "json_por_segmento": []
    }

    if not analisis_por_segmento_cot:
        continue # Saltar si no hay segmentos para este texto

    # Iterar por cada segmento
    total_segmentos_texto = len(analisis_por_segmento_cot)
    for k, info_cot_seg in enumerate(analisis_por_segmento_cot):
        segmentos_procesados_count += 1
        texto_segmento_actual = info_cot_seg.get("segmento_texto", "")
        cot_completo_segmento = info_cot_seg.get("analisis_cot_completo")
        error_previo_cot = info_cot_seg.get("error")

        # Actualizar progreso
        progreso_actual = segmentos_procesados_count / total_segmentos_global
        progress_bar.value = progreso_actual
        progress_label.value = f"{segmentos_procesados_count}/{total_segmentos_global} Segmentos ({errores_json_count} JSON err, {errores_api_count} API err)"


        # --- Lógica de generación JSON (simplificada en output) ---
        segmento_json_resultado = {
            "segmento_texto": texto_segmento_actual,
            "analisis_json_object": None, "error": None,
            "raw_response": None, "retries_count": 0
        }

        # Usar CoT o placeholder
        contexto_cot_para_prompt = cot_completo_segmento
        if error_previo_cot:
             contexto_cot_para_prompt = f"Análisis CoT no disponible (Error previo: {error_previo_cot})"
             segmento_json_resultado["error"] = "Dependencia CoT fallida" # Marcar error si depende de CoT
             resultados_json_final_revisado[id_texto]["json_por_segmento"].append(segmento_json_resultado)
             errores_json_count +=1 # Contar como error
             continue # Saltar al siguiente segmento si el CoT falló
        elif not cot_completo_segmento:
             contexto_cot_para_prompt = "Análisis CoT no disponible (Vacío)."
             # Decidir si continuar o marcar error - aquí continuamos pero el JSON podría ser malo
             # segmento_json_resultado["error"] = "Dependencia CoT vacía"
             # resultados_json_final_revisado[id_texto]["json_por_segmento"].append(segmento_json_resultado)
             # errores_json_count +=1
             # continue

        # Bucle de reintentos (sin prints internos detallados)
        json_valido = False
        intentos_json = 0
        while intentos_json <= MAX_JSON_RETRIES and not json_valido:
            try:
                prompt_json_revisado = f"""
Analiza lingüísticamente **cada palabra y signo de puntuación** presente en el "Segmento Original", utilizando el contexto proporcionado.
Contexto Principal (Análisis CoT Detallado Previo):
---
{contexto_cot_para_prompt}
---
Segmento Original a Analizar: "{texto_segmento_actual}"
Texto Completo Original (Referencia Adicional):
---
{texto_original_completo}
---
Tarea Principal: Tu respuesta DEBE SER **ÚNICA Y EXCLUSIVAMENTE** una lista JSON válida `[...]`, sin comentarios, texto introductorio/conclusivo, explicaciones ni marcadores de código ```json.
Para **CADA token** (palabra o signo de puntuación) en el "Segmento Original a Analizar", genera un objeto JSON (un diccionario `{...}`) siguiendo el schema apropiado para su categoría gramatical identificada en el **Paso 3 del CoT Detallado Previo**.
Schemas JSON por Categoría (Campos a incluir DENTRO de cada diccionario):
{schemas_json_string}
Instrucciones Específicas para Rellenar Campos: Utiliza el **Análisis CoT Detallado Previo** como guía principal para la interpretación, categoría gramatical, desambiguación y contexto. Rellena `"palabra_analizada"` con la forma exacta del token del segmento. Infiere `"lemma"` y rasgos morfológicos (género, número, tiempo, modo, etc.) basándote en el CoT y la gramática. Proporciona `"definicion_contextual"` clara y relevante para el segmento, derivada del CoT. Proporciona `"definicion_general"` estándar si aplica. Genera 1-2 `"ejemplos_uso"` **nuevos** y claros. Si un campo definido en el schema no aplica o no se puede determinar incluso con el CoT, usa `null`. Intenta omitir campos opcionales no relevantes si es posible.
Formato de Salida OBLIGATORIO: Genera **SOLAMENTE** la lista JSON `[...]`. Cada elemento de la lista debe ser un diccionario `{...}` representando un token analizado. La lista debe ser **PLANA** (sin anidamiento por categoría).
REPITO: Tu respuesta DEBE SER ÚNICAMENTE el array JSON.
"""
                generation_config_json = genai.types.GenerationConfig(temperature=TEMPERATURA_JSON)
                safety_settings = None # Definir si es necesario

                response_json = model.generate_content(prompt_json_revisado, generation_config=generation_config_json, safety_settings=safety_settings)

                raw_response_text_json = response_json.text
                cleaned_json_text = re.sub(r"^\s*```json\s*|\s*```\s*$", "", raw_response_text_json, flags=re.MULTILINE | re.DOTALL).strip()
                segmento_json_resultado["raw_response"] = cleaned_json_text

                try:
                    parsed_json_object = json.loads(cleaned_json_text)
                    if isinstance(parsed_json_object, list):
                        segmento_json_resultado["analisis_json_object"] = parsed_json_object
                        segmento_json_resultado["error"] = None
                        json_valido = True
                        # print(f"    ✅ JSON OK (Intento {intentos_json+1}).") # Comentado
                    else:
                        raise json.JSONDecodeError("La respuesta no es una lista JSON.", cleaned_json_text, 0)
                except json.JSONDecodeError as json_e:
                     segmento_json_resultado["error"] = f"JSONDecodeError (Intento {intentos_json+1}): {json_e}"
                     # No imprimir error aquí, se cuenta al final si fallan todos los reintentos

            except exceptions.ResourceExhausted as e_rate:
                # Imprimir solo el primer error de Rate Limit por segmento
                if intentos_json == 0: print(f"  ⚠️ Rate Limit en {id_texto}, Seg {k+1}. Esperando 60s...")
                segmento_json_resultado["error"] = f"ERROR API (Rate Limit): {e_rate}"
                errores_api_count+=1
                time.sleep(60); continue # Reintenta la llamada API, no cuenta como intento JSON
            except Exception as e_api:
                 # Imprimir solo el primer error de API por segmento
                 if intentos_json == 0: print(f"  ⛔ Error API en {id_texto}, Seg {k+1}: {e_api}")
                 segmento_json_resultado["error"] = f"ERROR API/Otro (Intento {intentos_json+1}): {e_api}"
                 errores_api_count+=1
                 # Considerar break si es un error grave

            intentos_json += 1
            segmento_json_resultado["retries_count"] = intentos_json

        # Fin del bucle while
        if not json_valido:
             # Imprimir error SOLO si fallaron todos los reintentos de parseo JSON
             print(f"  ❌ Falló JSON Parse para {id_texto}, Seg {k+1}. Último error: {segmento_json_resultado.get('error', 'Desconocido')}")
             errores_json_count += 1

        # Guardar resultado (con o sin JSON válido)
        resultados_json_final_revisado[id_texto]["json_por_segmento"].append(segmento_json_resultado)

        # Pausa CORTA entre segmentos si todo va bien, MÁS LARGA si hubo errores recientes
        pausa = 1 if json_valido and errores_api_count == 0 else 3 # Pausa corta si OK, más larga si hubo problemas
        time.sleep(pausa)

    # Fin del bucle por segmentos
    # print(f"   Análisis JSON completado para '{id_texto}'.") # Comentado

# --- Fin del Bucle por Textos ---
# Limpiar barra de progreso al final
time.sleep(0.5)
# progress_bar.close()
# progress_label.close()
# ¿O ocultarla?
# progress_widget.layout.display = 'none' # Si progress_widget es el VBox


print(f"\n--- Generación de Análisis JSON Completada ---")
print(f"Resumen: {segmentos_procesados_count} segmentos procesados.")
if errores_json_count > 0: print(f"  - {errores_json_count} errores de parseo JSON.")
if errores_api_count > 0: print(f"  - {errores_api_count} errores de API (Rate Limit u otros).")


--- Iniciando Generación de Análisis JSON Detallado por Palabra ---


  ⛔ Error API en Texto_16, Seg 4: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))

--- Generación de Análisis JSON Completada ---
Resumen: 98 segmentos procesados.
  - 1 errores de API (Rate Limit u otros).


In [111]:
# @title 7.A: Test
import pprint
if 'resultados_json_final_revisado' in globals() and resultados_json_final_revisado:
    print("Verificando 'resultados_json_final_revisado'...")
    first_text_id = list(resultados_json_final_revisado.keys())[0]
    print(f"Claves para {first_text_id}: {list(resultados_json_final_revisado[first_text_id].keys())}")
    if 'json_por_segmento' in resultados_json_final_revisado[first_text_id]:
        print(f"Número de segmentos JSON para {first_text_id}: {len(resultados_json_final_revisado[first_text_id]['json_por_segmento'])}")
        if resultados_json_final_revisado[first_text_id]['json_por_segmento']:
             print("Ejemplo del primer segmento JSON:")
             pprint.pprint(resultados_json_final_revisado[first_text_id]['json_por_segmento'][0], depth=2) # Mostrar solo 2 niveles de profundidad
             if 'analisis_json_object' in resultados_json_final_revisado[first_text_id]['json_por_segmento'][0]:
                  print("--> ¡Contiene 'analisis_json_object'!")
             else:
                  print("--> (!) NO contiene 'analisis_json_object'.")

    else:
         print("(!) Falta la clave 'json_por_segmento'.")

else:
    print("(!) 'resultados_json_final_revisado' no encontrado o vacío después de Celda 7.")


Verificando 'resultados_json_final_revisado'...
Claves para Texto_1: ['texto_original', 'tipo_segmentacion_usada', 'json_por_segmento']
Número de segmentos JSON para Texto_1: 1
Ejemplo del primer segmento JSON:
{'analisis_json_object': [{...},
                          {...},
                          {...},
                          {...},
                          {...},
                          {...},
                          {...},
                          {...},
                          {...},
                          {...},
                          {...}],
 'error': None,
 'raw_response': '[\n'
                 '  {\n'
                 '    "categoria": "pronombre",\n'
                 '    "palabra_analizada": "Me",\n'
                 '    "tipo": "personal",\n'
                 '    "persona": 1,\n'
                 '    "genero": null,\n'
                 '    "numero": "singular",\n'
                 '    "caso": "dativo",\n'
                 '    "tonicidad": "átono",

In [112]:
# @title 8: Visualizar Resultados Interactivamente por Segmento

import json
import ipywidgets as widgets
from IPython.display import display, Markdown, clear_output
import pprint # Usaremos pprint para el JSON por si json.dumps falla o queremos ver la estructura Python

# --- Verificaciones Previas ---
results_ok = True
if 'resultados_analisis_cot' not in globals() or not resultados_analisis_cot:
    print("⛔ ERROR: No se encontraron los resultados del CoT detallado ('resultados_analisis_cot'). Celda 6 necesaria.")
    results_ok = False
if 'resultados_json_final_revisado' not in globals() or not resultados_json_final_revisado:
    print("⛔ ERROR: No se encontraron los resultados del JSON final ('resultados_json_final_revisado'). Celda 7 necesaria.")
    results_ok = False

# --- Preparar Datos para Dropdowns ---
text_options = []
if results_ok:
    # Crear lista de tuplas (label_con_preview, id_texto) para el primer dropdown
    for id_texto, data in resultados_analisis_cot.items():
        texto_original = data.get("texto_original", "")
        preview = texto_original[:20].replace('\n', ' ') # Primeros 20 caracteres, sin saltos de línea
        label = f"{id_texto}: {preview}..."
        text_options.append((label, id_texto))

if not text_options:
    print("\n⚠️ No hay textos procesados con CoT para visualizar.")
    # Detener la creación de widgets si no hay opciones
    results_ok = False

# --- Crear Widgets ---
if results_ok:
    # Dropdown 1: Selección de Texto
    text_dd = widgets.Dropdown(
        options=text_options,
        description='Texto:',
        style={'description_width': 'initial'}, # Ajustar ancho de descripción
        layout={'width': 'max-content'} # Ajustar ancho del dropdown
    )

    # Dropdown 2: Selección de Segmento (se llenará dinámicamente)
    segment_dd = widgets.Dropdown(
        options=[], # Empieza vacío
        description='Segmento:',
        style={'description_width': 'initial'},
        layout={'width': 'max-content'},
        disabled=True # Empieza deshabilitado hasta que se elija un texto
    )

    # Área de Salida para mostrar resultados
    output_area = widgets.Output()

    # --- Funciones de Callback (Event Handlers) ---

    # Se ejecuta cuando cambia el texto seleccionado
    def on_text_change(change):
        if change['type'] == 'change' and change['name'] == 'value':
            selected_text_id = change['new']
            segment_options = [] # Reiniciar opciones de segmento
            segment_dd.options = [] # Limpiar dropdown
            segment_dd.value = None # Deseleccionar
            segment_dd.disabled = True # Deshabilitar mientras carga
            output_area.clear_output() # Limpiar resultados anteriores

            if selected_text_id and selected_text_id in resultados_analisis_cot:
                segmentos_info = resultados_analisis_cot[selected_text_id].get("analisis_por_segmento", [])
                if segmentos_info:
                    for idx, seg_info in enumerate(segmentos_info):
                        texto_preview = seg_info.get('segmento_texto', 'ERROR')[:80].replace('\n', ' ')
                        label = f"Segmento {idx + 1}: {texto_preview}..."
                        # El valor del dropdown será el índice (0, 1, 2...)
                        segment_options.append((label, idx))

                    segment_dd.options = segment_options
                    segment_dd.disabled = False # Habilitar dropdown de segmentos
                else:
                     with output_area:
                         print(f"No se encontraron segmentos analizados para '{selected_text_id}'.")
            else:
                 with output_area:
                      print(f"ID de texto '{selected_text_id}' no encontrado.")

    # Se ejecuta cuando cambia el segmento seleccionado
    def on_segment_change(change):
        if change['type'] == 'change' and change['name'] == 'value':
            selected_segment_index = change['new']
            selected_text_id = text_dd.value # Obtener el texto ID actual
            output_area.clear_output() # Limpiar antes de mostrar nuevos resultados

            if selected_text_id is None or selected_segment_index is None:
                # Esto puede pasar si se deselecciona o al inicio
                return

            with output_area:
                display(Markdown(f"### Resultados para '{selected_text_id}', Segmento {selected_segment_index + 1}"))
                display(Markdown("---"))

                # Mostrar CoT
                try:
                    data_cot_segmento = resultados_analisis_cot[selected_text_id]["analisis_por_segmento"][selected_segment_index]
                    texto_segmento_actual = data_cot_segmento.get("segmento_texto", "N/A")
                    cot_completo = data_cot_segmento.get("analisis_cot_completo")
                    error_cot = data_cot_segmento.get("error")

                    display(Markdown(f"**Segmento:** `{texto_segmento_actual}`"))
                    display(Markdown("---"))
                    display(Markdown("**Análisis CoT Detallado (Celda 6):**"))
                    if error_cot:
                        display(Markdown(f"```\nERROR al generar CoT:\n{error_cot}\n```"))
                    elif cot_completo:
                        display(Markdown(f"```markdown\n{cot_completo}\n```"))
                    else:
                        display(Markdown("*(No se encontró CoT generado)*"))

                except (IndexError, KeyError, TypeError) as e:
                    display(Markdown(f"❌ ERROR al acceder/mostrar datos CoT: {e}"))

                # Mostrar JSON
                try:
                    # Verificar si resultados_json_final_revisado está disponible
                    if 'resultados_json_final_revisado' in globals() and resultados_json_final_revisado and selected_text_id in resultados_json_final_revisado:
                        data_json_segmento = resultados_json_final_revisado[selected_text_id]["json_por_segmento"][selected_segment_index]
                        json_analizado = data_json_segmento.get("analisis_json_object")
                        error_json = data_json_segmento.get("error")
                        raw_response_json = data_json_segmento.get("raw_response")

                        display(Markdown("---"))
                        display(Markdown("**Análisis JSON por Palabra (Celda 7):**"))

                        if error_json:
                            display(Markdown(f"```\nERROR al generar/parsear JSON:\n{error_json}\n```"))
                            if raw_response_json:
                                 display(Markdown(f"**Respuesta Cruda Recibida:**\n```\n{raw_response_json[:1000]}...\n```"))
                        elif json_analizado:
                            # Usar json.dumps para formato JSON estándar
                            try:
                                json_string_pretty = json.dumps(json_analizado, indent=2, ensure_ascii=False)
                                display(Markdown(f"```json\n{json_string_pretty}\n```"))
                            except Exception as dump_err:
                                display(Markdown(f"⚠️ Error al formatear JSON con json.dumps: {dump_err}"))
                                display(Markdown("Mostrando estructura Python con pprint:"))
                                # Fallback a pprint si json.dumps falla por alguna razón rara
                                pprint.pprint(json_analizado, indent=2, width=120)

                        else:
                            display(Markdown("*(No se encontró JSON generado/parseado)*"))
                    else:
                         display(Markdown("*(Resultados JSON no disponibles para este texto)*"))

                except (IndexError, KeyError, TypeError) as e:
                    display(Markdown(f"❌ ERROR al acceder/mostrar datos JSON: {e}"))


    # --- Conectar Widgets y Funciones ---
    text_dd.observe(on_text_change, names='value')
    segment_dd.observe(on_segment_change, names='value')

    # --- Mostrar Widgets ---
    print("Selecciona un texto y luego un segmento para ver los detalles:")
    display(widgets.VBox([text_dd, segment_dd, output_area]))

    # --- Carga Inicial ---
    # Simular un cambio inicial para cargar los segmentos del primer texto
    on_text_change({'type': 'change', 'name': 'value', 'new': text_dd.value, 'old': None})

Selecciona un texto y luego un segmento para ver los detalles:


In [113]:
# @title 9: Generar CoT Global Jerárquico (Síntesis Equitativa por Niveles)

import math
import time
import json
from google.api_core import exceptions
# ... (resto de importaciones y verificaciones previas igual) ...

# --- Parámetros para la Síntesis Jerárquica ---
#@markdown ### Configuración de la Síntesis Jerárquica
#@markdown Tamaño del grupo **deseado** para sintetizar CoTs en cada nivel (se ajustará para distribución equitativa).
group_size_synthesis = 4 #@param {type:"integer"}
#@markdown Temperatura para la generación de CoT Global/Intermedio.
temperature_synthesis = 0.6 #@param {type:"slider", min:0.0, max:1.0, step:0.1}
#@markdown Máximos reintentos por llamada API de síntesis si falla.
max_retries_synthesis = 1 #@param {type:"integer"}
#@markdown Segundos de espera entre reintentos o ante Rate Limits.
sleep_time_synthesis = 45 #@param {type:"integer"}
#@markdown Máximo número de niveles de síntesis (prevención de bucles infinitos).
max_synthesis_levels = 10 #@param {type:"integer"}


print(f"\n--- Iniciando Generación de CoT Global Jerárquico (Distribución Equitativa) ---") # <-- Mensaje actualizado
print(f"Parámetros: group_size_target={group_size_synthesis}, temperature={temperature_synthesis}, max_retries={max_retries_synthesis}, sleep_time={sleep_time_synthesis}s")

# Diccionario para almacenar los CoT globales finales
resultados_cot_global = {}

# --- Función Auxiliar para la Síntesis de un Grupo ---
# La función synthesize_cot_group permanece IGUAL que en la versión anterior
# Solo necesita recibir la lista correcta de input_cots
def synthesize_cot_group(
    input_cots: list,
    original_text: str,
    level: int,
    chunk_index: int,
    total_chunks_at_level: int,
    model_to_use: genai.GenerativeModel,
    temp: float,
    retries: int,
    sleep_time: int,
    id_texto_logging: str
) -> str:
    """Genera un CoT de síntesis para un grupo de CoTs de nivel inferior."""

    if not input_cots:
        return f"ERROR INTERNO: No se proporcionaron CoTs de entrada para el nivel {level}, chunk {chunk_index+1}."

    # Combinar los CoTs de entrada con separadores claros
    combined_input_text = f"\n\n---\n".join(
        f"ANÁLISIS PREVIO {i+1}:\n{cot}" for i, cot in enumerate(input_cots)
    )

    num_inputs = len(input_cots)
    context_description = f"Estos {num_inputs} análisis provienen de un nivel previo de síntesis (Nivel {level-1})." if level > 1 else f"Estos {num_inputs} análisis detallados provienen directamente de los segmentos originales del texto."

    # Construir el prompt (adaptado del prompt global original)
    # AÑADIR INSTRUCCIÓN DE BALANCEO
    prompt_synthesis = f"""
**Rol:** Actúa como un analista lingüístico senior experto en síntesis y análisis del discurso.

**Tarea Principal:** Realiza un análisis lingüístico SINTÉTICO y de ALTO NIVEL basado **EXCLUSIVAMENTE** en los análisis previos proporcionados a continuación. Tu objetivo es extraer patrones, evaluar coherencia e interpretar el significado consolidado de la porción del texto cubierta por estos análisis, manteniendo siempre como referencia el texto completo original.

**Texto Original Completo (Referencia Contextual):**
```
{original_text}
```

**Contexto Clave (Análisis Previos a Sintetizar - {num_inputs} en total):**
{context_description}
--- INICIO ANÁLISIS PREVIOS ---
{combined_input_text}
--- FIN ANÁLISIS PREVIOS ---

**Proceso de Análisis Sintético (Sigue estos pasos rigurosamente y sin añadir saltos de línea innecesarios):**

**1. Consolidación del Contexto y Temática del Grupo:**
    *   **1.1. Contexto y Género Inferido (Consolidado):** Basándote en la consistencia/variación de los análisis previos, ¿cuál es la caracterización contextual (Tipo texto, Registro, Intención) y temática más probable para la sección del texto cubierta por *estos* análisis?
    *   **1.2. Ideas Centrales del Grupo:** Resume las ideas, temas o eventos clave presentados en los análisis proporcionados.

**2. Análisis Estructural y Coherencia Interna del Grupo:**
    *   **2.1. Organización Aparente:** ¿Sugieren los análisis una estructura o progresión particular dentro de esta sección del texto?
    *   **2.2. Coherencia y Cohesión Local:** ¿Qué tan coherentes y cohesionados parecen ser los segmentos analizados previamente, según la información dada? ¿Se identificaron mecanismos de cohesión relevantes o rupturas?

**3. Síntesis Semántica y Estilística del Grupo:**
    *   **3.1. Significado Consolidado:** ¿Cuál es el significado o efecto general que se desprende de la combinación de estos análisis? ¿Se resolvieron o mantuvieron ambigüedades mencionadas?
    *   **3.2. Estilo y Recursos Notables:** ¿Emergen patrones estilísticos o recursos retóricos dominantes de los análisis proporcionados para esta sección?

**4. Patrones Lingüísticos Relevantes en el Grupo:**
    *   **4.1. Léxico/Sintaxis Característicos:** ¿Se mencionan usos léxicos o patrones sintácticos recurrentes o destacables en los análisis previos?

**5. Evaluación Crítica Preliminar del Grupo:**
    *   **5.1. Complejidad/Claridad Aparente:** Según los análisis, ¿esta sección del texto parece ser lingüísticamente compleja o clara?
    *   **5.2. Puntos Clave para Niveles Superiores:** Resume en 1-2 puntos los hallazgos más cruciales de esta síntesis que deberían ser considerados al integrar este resultado con otros.

**Instrucciones Finales:**
*   Sé conciso pero completo. Enfócate en **sintetizar** la información dada.
*   Basa tus conclusiones **únicamente** en los "Análisis Previos" proporcionados y el "Texto Original Completo".
*   **IMPORTANTE:** Asegúrate de que tu síntesis dé una **consideración equilibrada** a la información proveniente de cada uno de los {num_inputs} análisis previos al identificar patrones y conclusiones generales para este grupo.
*   Adopta un tono analítico y objetivo.
*   Tu respuesta debe ser el análisis sintético siguiendo los 5 pasos. No incluyas el prompt original en la respuesta.
""" # <-- Fin del prompt actualizado

    generation_config_synth = genai.types.GenerationConfig(temperature=temp)
    safety_settings = None # Reutilizar config global si existe o definir aquí

    current_retry = 0
    while current_retry <= retries:
        try:
            print(f"    -> Llamando API para síntesis (Texto: {id_texto_logging}, Nivel {level}, Chunk {chunk_index+1}/{total_chunks_at_level}, Items: {num_inputs}, Intento {current_retry+1}/{retries+1})...") # <-- Log más detallado
            response_synth = model_to_use.generate_content(
                prompt_synthesis,
                generation_config=generation_config_synth,
                safety_settings=safety_settings
            )
            synthesis_output = response_synth.text.strip()
            # Simple validación anti-vacío
            if not synthesis_output:
                 raise ValueError("La respuesta de la API estaba vacía.")
            print(f"    <- Síntesis recibida (Nivel {level}, Chunk {chunk_index+1}).")
            return synthesis_output

        except exceptions.ResourceExhausted as e_rate:
            print(f"    ⚠️ ERROR API (Rate Limit) en Síntesis (Nivel {level}, Chunk {chunk_index+1}, Intento {current_retry+1}): {e_rate}. Esperando {sleep_time}s...")
            time.sleep(sleep_time)
            # No incrementar retry count aquí, la espera es el manejo
            continue # Vuelve a intentar la llamada
        except Exception as e_synth:
            print(f"    ⛔ Error generando Síntesis (Nivel {level}, Chunk {chunk_index+1}, Intento {current_retry+1}): {e_synth}")
            current_retry += 1
            if current_retry <= retries:
                print(f"       Reintentando en {sleep_time/2:.1f} segundos...")
                time.sleep(sleep_time / 2) # Espera más corta entre reintentos normales
            else:
                error_message = f"ERROR: Falló la síntesis para Nivel {level}, Chunk {chunk_index+1} después de {retries+1} intentos. Último error: {e_synth}"
                print(f"    ❌ {error_message}")
                return error_message # Devolver mensaje de error como resultado

    # Si se agotan los reintentos
    final_error_message = f"ERROR: Se agotaron los reintentos ({retries+1}) para la síntesis (Nivel {level}, Chunk {chunk_index+1})."
    print(f"    ❌ {final_error_message}")
    return final_error_message


# --- Bucle Principal por Texto ---
total_textos_a_procesar = len(resultados_analisis_cot)
textos_procesados_count = 0
for id_texto, data_texto in resultados_analisis_cot.items():
    textos_procesados_count += 1
    print(f"\n L ({textos_procesados_count}/{total_textos_a_procesar}) Procesando CoT Global para: {id_texto}")

    texto_original = data_texto["texto_original"]
    segment_analysis_list = data_texto.get("analisis_por_segmento", [])

    # Extraer solo los CoT detallados de los segmentos (Nivel 0)
    current_level_cots = []
    for i, seg_info in enumerate(segment_analysis_list):
        cot = seg_info.get("analisis_cot_completo")
        if cot and not seg_info.get("error"): # Solo añadir si no hubo error previo y hay CoT
            current_level_cots.append(cot)
        else:
            error_msg = seg_info.get("error", "Error desconocido en CoT de segmento")
            placeholder_cot = f"--- ANÁLISIS FALLIDO O VACÍO PARA SEGMENTO {i+1} ---\nError reportado: {error_msg}\n--- FIN ERROR ---"
            current_level_cots.append(placeholder_cot)
            print(f"  ⚠️ Usando placeholder para CoT fallido/vacío del segmento {i+1}.")

    if not current_level_cots:
        print(f"  ❌ No se encontraron CoT de segmentos válidos o placeholders para '{id_texto}'. Saltando CoT Global.")
        resultados_cot_global[id_texto] = {
            "texto_original": texto_original,
            "cot_global_final": None,
            "error": "No CoT de segmentos para procesar.",
            "niveles_completados": 0,
            "log_proceso": ["Inicio: No CoT de segmentos."]
        }
        continue

    # --- Bucle de Síntesis Jerárquica (LÓGICA DE CHUNKING MODIFICADA) ---
    level_num = 0
    process_log = []
    start_time_text = time.time()

    while len(current_level_cots) > 1 and level_num < max_synthesis_levels:
        level_num += 1
        num_items_current_level = len(current_level_cots)
        print(f"  --- Iniciando Nivel de Síntesis {level_num} ({num_items_current_level} CoTs a agrupar) ---")
        process_log.append(f"Nivel {level_num}: Iniciando con {num_items_current_level} CoTs.")

        # --- Inicio: Lógica de Distribución Equitativa ---
        target_group_size = group_size_synthesis
        num_items_current_level = len(current_level_cots) # Aseguramos tener el número actual

        if num_items_current_level <= 1:
             # Esto no debería ocurrir aquí debido a la condición del while, pero es una guarda segura.
             # Si solo queda 1 o 0, no hay chunks que crear en este nivel. El bucle while terminará.
             num_chunks = 0 # O podríamos hacer break, pero num_chunks=0 lo maneja el siguiente if.
        elif 1 < num_items_current_level <= target_group_size:
            # *** LA CORRECCIÓN CLAVE ESTÁ AQUÍ ***
            # Si quedan más de 1 pero caben todos en un solo grupo (o son menos que el tamaño del grupo),
            # entonces SÓLO creamos UN chunk para realizar la fusión final.
            num_chunks = 1
        else:
            # Si hay más elementos que el tamaño del grupo deseado, calculamos
            # el número de chunks para una distribución equitativa.
            num_chunks = math.ceil(num_items_current_level / target_group_size)

        # Comprobación de seguridad por si algo raro pasa
        if num_chunks == 0 and num_items_current_level > 0 :
             print(f"  ❌ Error interno: num_chunks es 0 pero quedan {num_items_current_level} items en nivel {level_num}. Deteniendo para este texto.")
             process_log.append(f"Error: num_chunks=0 con {num_items_current_level} items.")
             # Forzar salida del bucle while para este texto
             current_level_cots = [f"ERROR: Cálculo de chunks falló en nivel {level_num}"]
             break
        elif num_chunks > 0:
             # Calculamos tamaño base y residuo SOLO si vamos a crear chunks
             base_chunk_size = num_items_current_level // num_chunks
             remainder = num_items_current_level % num_chunks
        else:
             # Si num_chunks es 0 (porque num_items <= 1), no necesitamos estos valores
             base_chunk_size = 0
             remainder = 0
        # --- Fin: Lógica de Distribución Equitativa ---

        next_level_cots = []
        current_index = 0 # Índice para recorrer current_level_cots

        # Iterar por el número calculado de chunks
        for i in range(num_chunks):
            # Determinar tamaño de este chunk específico
            current_chunk_size = base_chunk_size + 1 if i < remainder else base_chunk_size

            # Asegurar que no intentemos tomar más elementos de los que quedan
            if current_index + current_chunk_size > num_items_current_level:
                # Esto no debería ocurrir con la lógica correcta, pero es una salvaguarda
                print(f"  ⚠️ Advertencia: Ajustando tamaño del último chunk {i+1} para que coincida con los elementos restantes.")
                current_chunk_size = num_items_current_level - current_index
                if current_chunk_size <= 0: continue # Si ya no quedan, saltar


            # Obtener los CoTs para este chunk
            chunk_cots = current_level_cots[current_index : current_index + current_chunk_size]
            log_chunk_start = f"  - Procesando Chunk {i+1}/{num_chunks} (Nivel {level_num}) con {len(chunk_cots)} CoTs (Índices {current_index}-{current_index + len(chunk_cots) - 1})."
            print(log_chunk_start)
            process_log.append(log_chunk_start)

            # Llamar a la función de síntesis
            synthesized_cot = synthesize_cot_group(
                input_cots=chunk_cots,
                original_text=texto_original,
                level=level_num,
                chunk_index=i, # Usamos i como índice del chunk
                total_chunks_at_level=num_chunks,
                model_to_use=model,
                temp=temperature_synthesis,
                retries=max_retries_synthesis,
                sleep_time=sleep_time_synthesis,
                id_texto_logging=id_texto
            )
            next_level_cots.append(synthesized_cot)

            # Actualizar el índice para el próximo chunk
            current_index += len(chunk_cots) # Usar len(chunk_cots) real por si hubo ajuste

            # Pausa entre llamadas a chunks
            time.sleep(max(1, sleep_time_synthesis // 15)) # Pausa corta, ej: 3s si sleep_time=45

        # Actualizar la lista de CoTs para el siguiente nivel
        current_level_cots = next_level_cots
        log_level_end = f"  --- Fin Nivel {level_num}. Generados {len(current_level_cots)} CoTs de síntesis. ---"
        print(log_level_end)
        process_log.append(log_level_end)
        # Pausa un poco más larga entre niveles completos
        time.sleep(max(2, sleep_time_synthesis // 10)) # Ej: 4.5s si sleep_time=45

    # --- Fin del Bucle Jerárquico ---
    # ... (El resto del código para manejar el resultado final y guardarlo es IGUAL) ...
    end_time_text = time.time()
    duration_text = end_time_text - start_time_text

    # Guardar el resultado final
    final_cot_result = None
    final_error = None

    if level_num >= max_synthesis_levels and len(current_level_cots) > 1:
        final_error = f"Proceso detenido: Se alcanzó el máximo de {max_synthesis_levels} niveles de síntesis. Quedaron {len(current_level_cots)} CoTs sin consolidar."
        print(f"  ❌ {final_error}")
        final_cot_result = "\n\n---\n".join(current_level_cots) + f"\n\n*** ERROR: {final_error} ***"
    elif len(current_level_cots) == 1:
        final_cot_result = current_level_cots[0]
        if final_cot_result.strip().startswith("ERROR:"):
             final_error = f"El CoT Global final parece ser un mensaje de error propagado o falló la última síntesis: {final_cot_result[:200]}..."
             print(f"  ⚠️ {final_error}")
        elif "ANÁLISIS FALLIDO" in final_cot_result and len(segment_analysis_list) == 1:
             final_error = "El único segmento original falló, no se pudo generar CoT Global."
             print(f"  ❌ {final_error}")
             final_cot_result = None # No hay CoT válido
        else:
             print(f"  ✅ CoT Global Final generado exitosamente para '{id_texto}' en {duration_text:.2f} segundos.")
    elif not current_level_cots: # Si algo falló catastróficamente
         final_error = "Error catastrófico: No quedaron CoTs al final del proceso."
         print(f"  ❌ {final_error}")
    else: # Caso inesperado (ej: 0 CoTs por error en cálculo de chunks)
         final_error = f"Estado inesperado al final: {len(current_level_cots)} CoTs restantes (posiblemente mensajes de error)."
         print(f"  ❌ {final_error}")
         final_cot_result = "\n\n---\n".join(current_level_cots) + f"\n\n*** ERROR: {final_error} ***"


    resultados_cot_global[id_texto] = {
        "texto_original": texto_original,
        "cot_global_final": final_cot_result,
        "error": final_error,
        "niveles_completados": level_num,
        "log_proceso": process_log,
        "tiempo_procesamiento_seg": duration_text
    }

print("\n--- Generación de CoT Global Jerárquico (Distribución Equitativa) Completada ---") # <-- Mensaje actualizado

# Opcional: Mostrar el resultado del primer texto procesado
# ... (El código de visualización es IGUAL) ...
import pprint
if resultados_cot_global:
    primer_id_global = list(resultados_cot_global.keys())[0]
    print(f"\n--- Ejemplo de Resultado CoT Global para: {primer_id_global} ---")
    resultado_ejemplo = resultados_cot_global[primer_id_global]

    print(f"Texto Original (inicio): {resultado_ejemplo['texto_original'][:100]}...")
    print(f"Niveles Completados: {resultado_ejemplo['niveles_completados']}")
    print(f"Tiempo: {resultado_ejemplo.get('tiempo_procesamiento_seg', 'N/A'):.2f}s")
    if resultado_ejemplo['error']:
        print(f"\nError Reportado:\n{resultado_ejemplo['error']}")
    print("\n--- CoT Global Final ---")
    if resultado_ejemplo['cot_global_final']:
        # Usar Markdown para mejor visualización si es posible en el entorno
        try:
            from IPython.display import display, Markdown
            display(Markdown(f"```markdown\n{resultado_ejemplo['cot_global_final']}\n```"))
        except ImportError:
             print(resultado_ejemplo['cot_global_final']) # Fallback a print normal
    else:
        print("(No se generó CoT Global Final o falló)")

    # Opcional: Mostrar el log del proceso
    # print("\n--- Log del Proceso ---")
    # pprint.pprint(resultado_ejemplo['log_proceso'])

else:
    print("\nNo se generaron resultados de CoT Global.")


--- Iniciando Generación de CoT Global Jerárquico (Distribución Equitativa) ---
Parámetros: group_size_target=4, temperature=0.6, max_retries=1, sleep_time=45s

 L (1/19) Procesando CoT Global para: Texto_1
  ✅ CoT Global Final generado exitosamente para 'Texto_1' en 0.00 segundos.

 L (2/19) Procesando CoT Global para: Texto_2
  ✅ CoT Global Final generado exitosamente para 'Texto_2' en 0.00 segundos.

 L (3/19) Procesando CoT Global para: Texto_3
  ✅ CoT Global Final generado exitosamente para 'Texto_3' en 0.00 segundos.

 L (4/19) Procesando CoT Global para: Texto_4
  ✅ CoT Global Final generado exitosamente para 'Texto_4' en 0.00 segundos.

 L (5/19) Procesando CoT Global para: Texto_5
  ✅ CoT Global Final generado exitosamente para 'Texto_5' en 0.00 segundos.

 L (6/19) Procesando CoT Global para: Texto_6
  ✅ CoT Global Final generado exitosamente para 'Texto_6' en 0.00 segundos.

 L (7/19) Procesando CoT Global para: Texto_7
  ✅ CoT Global Final generado exitosamente para 'Texto

```markdown
**Análisis Lingüístico del Segmento: 'Me gustaría cortarle el pelo, ya está muy largo'**

**1. Reflexión Inicial y Contextualización:**

*   **1.1. Contexto Inferido:** El segmento parece pertenecer a un texto de tipo **conversacional** o **narrativo** con un registro **informal**. La expresión "Me gustaría" sugiere una expresión de deseo o intención personal, lo que es típico de la conversación cotidiana o de la narración en primera persona. La frase "ya está muy largo" refuerza esta impresión, ya que se refiere a una observación directa y personal sobre el estado del cabello de alguien. La intención pragmática dominante parece ser **expresar un deseo o intención relacionada con una acción futura**. No se detectan indicios de lenguaje poético o técnico.
*   **1.2. Ambigüedades Potenciales Preliminares:** La principal ambigüedad potencial reside en el pronombre "le". No se sabe a quién se refiere "le". También podría haber una ambigüedad menor en el significado de "largo", que podría referirse a la longitud física del cabello o, en un sentido figurado, a que la persona ha tenido el mismo corte de pelo durante mucho tiempo.
*   **1.3. Significado e Interpretación Hipotética:** El significado literal principal es que el hablante desea cortar el pelo de alguien porque considera que está demasiado largo. El significado implícito podría ser que el hablante se ofrece a cortar el pelo, o que está expresando su opinión sobre el aspecto de la persona.

**2. Estructura Básica del Segmento:**

*   **2.1. Tipo de Estructura:** El segmento consta de una **oración compuesta coordinada yuxtapuesta**. Está formada por dos oraciones simples unidas sin un nexo explícito (conjunción).
*   **2.2. Componentes Sintácticos/Ideacionales Clave:**
    *   Primera oración: "Me gustaría cortarle el pelo"
        *   Sujeto: Omitido (Yo)
        *   Núcleo Verbal: "gustaría" (forma conjugada del verbo "gustar")
        *   Complemento Indirecto: "Me"
        *   Complemento Directo: "cortarle el pelo"
            *   Núcleo: "cortar" (verbo en infinitivo)
            *   Complemento Indirecto: "le"
            *   Complemento Directo: "el pelo"
    *   Segunda oración: "ya está muy largo"
        *   Sujeto: Omitido (El pelo)
        *   Núcleo Verbal: "está" (forma conjugada del verbo "estar")
        *   Atributo: "muy largo"
            *   Adverbio: "muy"
            *   Adjetivo: "largo"

**3. Análisis Léxico-Gramatical Contextualizado:**

*   'Me': [Ambig. Resuelta] | [pronombre] | Pronombre personal átono, función de complemento indirecto.
*   'gustaría': [Ambig. Resuelta] | [verbo] | Forma condicional del verbo "gustar". Expresa un deseo o intención.
*   'cortarle': [Ambig. Resuelta] | [verbo] | Infinitivo "cortar" con el pronombre "le" enclítico.
*   'el': [Ambig. Resuelta] | [determinante] | Artículo determinado masculino singular.
*   'pelo': [Ambig. Resuelta] | [sustantivo] | Sustantivo común, masculino singular.
*   ',': [Ambig. Resuelta] | [puntuacion] | Signo de puntuación que indica una pausa breve y separa las dos oraciones coordinadas.
*   'ya': [Ambig. Resuelta] | [adverbio] | Adverbio de tiempo. En este contexto, indica que la situación descrita es actual o que ha llegado a un punto determinado.
*   'está': [Ambig. Resuelta] | [verbo] | Forma conjugada del verbo "estar". En este contexto, funciona como verbo copulativo.
*   'muy': [Ambig. Resuelta] | [adverbio] | Adverbio de cantidad que modifica al adjetivo "largo".
*   'largo': [Ambig. Resuelta] | [adjetivo] | Adjetivo calificativo que describe la longitud del pelo.
*   '.': [Ambig. Resuelta] | [puntuacion] | Signo de puntuación que indica el final de la oración.

**4. Revisión Crítica y Resolución de Conflictos:**

*   **4.1. Consistencia Interna:** Los hallazgos son coherentes. El análisis léxico-gramatical confirma la hipótesis inicial sobre el contexto conversacional y la función de expresar un deseo. La estructura de oración compuesta coordinada yuxtapuesta es típica del habla informal.
*   **4.2. Resolución Definitiva de Ambigüedades:** La ambigüedad del pronombre "le" *persiste*. El análisis sintáctico indica que es un complemento indirecto, pero no revela a quién se refiere. El contexto más amplio del texto original sería necesario para resolver esta ambigüedad referencial. La ambigüedad sobre el significado de "largo" se ha resuelto en el sentido de longitud física del cabello, dado el contexto general de la frase.
*   **4.3. Aspectos Lingüísticos Destacables:** El uso del condicional ("gustaría") en lugar del presente ("me gusta") suaviza la expresión del deseo, haciéndola más cortés y menos impositiva. La elipsis del sujeto en ambas oraciones es común en español y contribuye a la fluidez del lenguaje conversacional.

**5. Síntesis Lingüística Final:**

El segmento "Me gustaría cortarle el pelo, ya está muy largo" es una oración compuesta coordinada yuxtapuesta, típica del lenguaje conversacional informal, que expresa un deseo o intención relacionada con el corte de pelo de alguien. La ambigüedad referencial del pronombre "le" persiste, pero la estructura y el léxico sugieren una función de expresión de un deseo personal con un cierto grado de cortesía.
```

In [114]:
# @title 10: Visualizar CoT Global por Texto

import ipywidgets as widgets
from IPython.display import display, Markdown, clear_output
import textwrap # Para limitar el ancho del texto original si es muy largo

# --- Verificaciones Previas ---
results_ok = True
if 'resultados_cot_global' not in globals() or not resultados_cot_global:
    print("⛔ ERROR: No se encontraron los resultados del CoT Global jerárquico ('resultados_cot_global'). Celda 9 necesaria.")
    results_ok = False

# --- Preparar Datos para Dropdown ---
text_global_cot_options = []
if results_ok:
    # Crear lista de tuplas (label_con_preview, id_texto) para el dropdown
    for id_texto, data in resultados_cot_global.items():
        texto_original = data.get("texto_original", "")
        # Preview más corto para este dropdown
        preview = texto_original[:40].replace('\n', ' ')
        label = f"{id_texto}: {preview}..."
        text_global_cot_options.append((label, id_texto))

if not text_global_cot_options:
    print("\n⚠️ No hay textos con CoT Global procesados para visualizar.")
    # Detener la creación de widgets si no hay opciones
    results_ok = False

# --- Crear Widgets ---
if results_ok:
    # Dropdown: Selección de Texto para ver CoT Global
    text_global_cot_dd = widgets.Dropdown(
        options=text_global_cot_options,
        description='Texto:',
        style={'description_width': 'initial'},
        layout={'width': 'max-content', 'min_width': '300px'} # Asegurar ancho mínimo
    )

    # Área de Salida para mostrar el CoT Global
    global_cot_output_area = widgets.Output()

    # --- Funciones de Callback (Event Handlers) ---

    # Se ejecuta cuando cambia el texto seleccionado en el dropdown
    def on_global_cot_text_change(change):
        if change['type'] == 'change' and change['name'] == 'value':
            selected_text_id = change['new']
            global_cot_output_area.clear_output(wait=True) # Limpiar resultados anteriores con wait

            if not selected_text_id:
                with global_cot_output_area:
                    print("Por favor, selecciona un texto.")
                return

            # Acceder a los datos del CoT Global para el texto seleccionado
            if selected_text_id in resultados_cot_global:
                data_global = resultados_cot_global[selected_text_id]
                texto_original_completo = data_global.get("texto_original", "N/A")
                cot_global = data_global.get("cot_global_final")
                error_global = data_global.get("error")
                niveles = data_global.get("niveles_completados", "N/A")
                tiempo_seg = data_global.get("tiempo_procesamiento_seg", "N/A")

                with global_cot_output_area:
                    display(Markdown(f"### CoT Global para: `{selected_text_id}`"))
                    display(Markdown("---"))
                    display(Markdown(f"**Texto Original (Inicio):**"))
                    # Usar textwrap para mostrar el inicio del texto original de forma legible
                    display(Markdown(f"```\n{textwrap.fill(texto_original_completo[:500], width=80)}\n... (Mostrar más si es necesario) ...\n```"))
                    display(Markdown("---"))
                    display(Markdown(f"**Información del Proceso (Celda 9):**"))
                    display(Markdown(f"- Niveles de Síntesis: `{niveles}`"))
                    display(Markdown(f"- Tiempo de Procesamiento: `{tiempo_seg:.2f}s`" if isinstance(tiempo_seg, float) else f"`{tiempo_seg}`"))

                    if error_global:
                         display(Markdown(f"**Error Reportado durante la generación del CoT Global:**"))
                         display(Markdown(f"```\n{error_global}\n```"))
                         # A veces el CoT puede contener el error, mostrarlo si existe
                         if cot_global and error_global in cot_global:
                             display(Markdown(f"**Contenido Parcial / Error en CoT:**"))
                             display(Markdown(f"```markdown\n{cot_global}\n```"))
                         elif cot_global:
                             display(Markdown(f"**CoT (Puede estar incompleto o ser placeholder):**"))
                             display(Markdown(f"```markdown\n{cot_global}\n```"))

                    elif cot_global:
                        display(Markdown(f"**Resultado del CoT Global Final:**"))
                        # Mostrar como bloque de markdown para formato
                        display(Markdown(f"```markdown\n{cot_global}\n```"))
                    else:
                        display(Markdown("**Resultado del CoT Global Final:**"))
                        display(Markdown("*(No se generó o no está disponible)*"))

            else:
                 with global_cot_output_area:
                    print(f"ERROR: ID de texto '{selected_text_id}' no encontrado en los resultados de CoT Global.")

    # --- Conectar Widgets y Funciones ---
    text_global_cot_dd.observe(on_global_cot_text_change, names='value')

    # --- Mostrar Widgets ---
    print("Selecciona un texto para ver su CoT Global Final (generado en Celda 9):")
    display(widgets.VBox([text_global_cot_dd, global_cot_output_area]))

    # --- Carga Inicial ---
    # Simular un cambio inicial para cargar el CoT del primer texto en la lista
    if text_global_cot_dd.options: # Asegurarse de que hay opciones antes de intentar cargar
        on_global_cot_text_change({'type': 'change', 'name': 'value', 'new': text_global_cot_dd.value, 'old': None})

Selecciona un texto para ver su CoT Global Final (generado en Celda 9):


In [115]:
# @title 11: Embeddings Palabras y Segmentos + Fusión Datos

import google.generativeai as genai
import time
import numpy as np
import pandas as pd
import re
from google.api_core import exceptions
import textwrap
import json # Necesario para la fusión
from IPython.display import display # Para mostrar DataFrames

print("--- Celda 11 (Revisado): Embeddings Palabras/Segmentos y Fusión ---")

# --- Verificaciones Previas ---
if 'model' not in globals() or model is None: raise ValueError("Modelo Gemini no inicializado (Celda 2).")
if 'resultados_analisis_cot' not in globals() or not resultados_analisis_cot: raise NameError("Resultados CoT detallado no encontrados (Celda 6).")
if 'resultados_json_final_revisado' not in globals() or not resultados_json_final_revisado: raise NameError("Resultados JSON no encontrados (Celda 7).")

# --- Configuración General Embeddings ---
# @markdown Selecciona el ID del modelo de embedding (sin 'models/'):
embedding_model_id = "text-embedding-004" # @param ["text-embedding-004", "embedding-001"] {allow-input: true}
embedding_model_name_api = f"models/{embedding_model_id}" # Nombre completo para API

# @markdown Parámetros comunes (se usarán para ambos niveles, ajustar si es necesario):
batch_size_common = 100
sleep_time_common = 10

# --- Configuración Específica: SEGMENTOS ---
embedding_task_type_segment = "RETRIEVAL_DOCUMENT" #@param ["RETRIEVAL_DOCUMENT", "CLUSTERING", "SEMANTIC_SIMILARITY"]
max_input_tokens_segment = 2000
max_cot_extract_chars_segment = 500 # Para extraer resumen CoT

# --- Configuración Específica: PALABRAS ---
embedding_task_type_word = "SEMANTIC_SIMILARITY" #@param ["RETRIEVAL_DOCUMENT", "RETRIEVAL_QUERY", "SEMANTIC_SIMILARITY", "CLASSIFICATION", "CLUSTERING"]
max_input_tokens_word = 500

# --- Funciones Auxiliares (Reutilizadas/Adaptadas) ---

# Función para extraer resumen de CoT de SEGMENTO (Paso 5)
def extract_segment_cot_summary(cot_text: str, max_chars: int) -> str:
    if not cot_text or not isinstance(cot_text, str): return "(Análisis CoT no disponible)"
    # Buscar Paso 5 (más flexible)
    pattern1 = r"^(?:\s*[*]*\s*)?5\.(?:\s*[*]*\s*)?(?:Evaluación Crítica Preliminar|Síntesis Lingüística Final)?:?\s*(.*?)(?=\n(?:\s*[*]*\s*)?[6-9]\.|\Z)"
    pattern2 = r"^(?:\s*[*]*\s*)?5\.(?:\s*[*]*\s*)?(.*?)(?=\n(?:\s*[*]*\s*)?[6-9]\.|\Z)"
    match = None; summary = None
    for pattern in [pattern1, pattern2]:
        match = re.search(pattern, cot_text, re.MULTILINE | re.DOTALL | re.IGNORECASE)
        if match: summary = match.group(1).strip(); break
    if summary: return summary[:max_chars] + ('...' if len(summary) > max_chars else '')
    # Fallback si no encuentra Paso 5
    lines = cot_text.strip().split('\n'); fallback = "\n".join(lines[-3:]).strip()
    return fallback[-max_chars:] + ('...' if len(fallback) > max_chars else '')

# Función para preparar input de SEGMENTO
def prepare_segment_input(segment_text: str, segment_cot: str, max_len: int, max_cot_chars: int) -> str:
    if not segment_text: segment_text = "(Segmento vacío)"
    cot_extract = extract_segment_cot_summary(segment_cot, max_cot_chars)
    base_input = f"Segmento: \"{segment_text}\"\n\nAnálisis Clave: {cot_extract}"
    # Truncamiento básico (igual que en Celda 11 original)
    estimated_tokens = len(base_input) / 4
    if estimated_tokens <= max_len: return base_input
    else:
        allowed_chars = int(max_len * 3.5); overhead = len(f"Segmento: \"\"\n\nAnálisis Clave: ")
        allowed_chars -= overhead; segment_chars_to_keep = min(len(segment_text), allowed_chars)
        remaining_chars = allowed_chars - segment_chars_to_keep; cot_chars_to_keep = min(len(cot_extract), remaining_chars)
        truncated_segment = segment_text[:segment_chars_to_keep] + ('...' if len(segment_text) > segment_chars_to_keep else '')
        truncated_cot = cot_extract[:cot_chars_to_keep] + ('...' if len(cot_extract) > cot_chars_to_keep else '')
        return f"Segmento: \"{truncated_segment}\"\n\nAnálisis Clave: {truncated_cot}"

# Función para preparar input de PALABRA (igual que en 11.A)
def prepare_word_input(word_data: dict, max_len: int) -> str:
    word = word_data.get('palabra_analizada', word_data.get('texto', '?'))
    category = word_data.get('categoria', 'desconocida')
    definition = word_data.get('definicion_contextual', word_data.get('definicion_general', ''))
    lemma = word_data.get('lemma', word_data.get('infinitivo'))
    context_parts = [f"Palabra: {word}"]
    if category != 'desconocida': context_parts.append(f"Categoría: {category}")
    if lemma and lemma != word: context_parts.append(f"Lema: {lemma}")
    if definition: context_parts.append(f"Def: {textwrap.shorten(definition, width=50, placeholder='...')}")
    if word_data.get('genero'): context_parts.append(f"G: {word_data['genero']}")
    if word_data.get('numero'): context_parts.append(f"N: {word_data['numero']}")
    full_input = " | ".join(context_parts)
    max_chars_allowed = int(max_len * 3.5)
    if len(full_input) > max_chars_allowed: return full_input[:max_chars_allowed] + "..."
    else: return full_input

# --- Función Genérica para Generar Embeddings en Lotes ---
def get_embeddings_batch(inputs: list, identifiers: list, model_name: str, task_type: str, batch_size: int, sleep_time: int, item_type: str) -> list:
    """Genera embeddings en lotes y devuelve una lista de resultados (dicts)."""
    if not inputs: print(f"(!) No hay inputs válidos para embeddings de {item_type}."); return []

    print(f"\nIniciando generación para {len(inputs)} {item_type}s (Modelo: {model_name}, Task: {task_type})...")
    results_list = []
    api_errors_count = 0
    total_batches = (len(inputs) + batch_size - 1) // batch_size

    for i in range(0, len(inputs), batch_size):
        batch_inputs = inputs[i : i + batch_size]
        batch_ids = identifiers[i : i + batch_size]
        batch_num = (i // batch_size) + 1
        print(f"  Procesando Lote {item_type.capitalize()} {batch_num}/{total_batches}...")
        try:
            response = genai.embed_content(model=model_name, content=batch_inputs, task_type=task_type)
            if 'embedding' in response and isinstance(response['embedding'], list) and len(response['embedding']) == len(batch_inputs):
                # print(f"    ✅ Lote {batch_num} OK.")
                for j, identifier in enumerate(batch_ids):
                     results_list.append({**identifier, 'embedding': np.array(response['embedding'][j]), 'error': None})
            else:
                 print(f"    ⛔ Error Lote {batch_num}: Respuesta API inesperada."); api_errors_count += len(batch_ids)
                 for identifier in batch_ids: results_list.append({**identifier, 'embedding': None, 'error': 'Respuesta API inesperada'})
        except exceptions.ResourceExhausted as e_rate:
            print(f"    ⚠️ Rate Limit Lote {batch_num}. Esperando {sleep_time}s..."); time.sleep(sleep_time); print(f"    ⛔ Lote {batch_num} falló."); api_errors_count += len(batch_ids)
            for identifier in batch_ids: results_list.append({**identifier, 'embedding': None, 'error': f'RateLimitError: {e_rate}'})
        except Exception as e:
            print(f"    ⛔ Error procesando Lote {batch_num}: {e}"); api_errors_count += len(batch_ids)
            for identifier in batch_ids: results_list.append({**identifier, 'embedding': None, 'error': str(e)})
        if batch_num < total_batches: time.sleep(max(0.5, sleep_time / 10))

    print(f"-> Generación {item_type}s completada. {len(results_list) - api_errors_count} embeddings OK, {api_errors_count} errores API.")
    return results_list

# ======================================================================
# --- PASO 1: Generar Embeddings y DataFrame Base para SEGMENTOS ---
# ======================================================================
print("\n\n--- PASO 1: Generando Embeddings para SEGMENTOS ---")
all_segment_inputs = []
segment_identifiers = [] # Lista de dicts {'texto_id': ..., 'segmento_idx': ...}

print("Preparando inputs para embeddings de segmentos...")
for texto_id, data_texto in resultados_analisis_cot.items():
    for seg_idx, info_cot_seg in enumerate(data_texto.get("analisis_por_segmento", [])):
         segmento_texto_actual = info_cot_seg.get("segmento_texto", "")
         cot_completo_segmento = info_cot_seg.get("analisis_cot_completo")
         # Incluir incluso si falta texto o CoT, la función prepare lo maneja
         input_str = prepare_segment_input(segmento_texto_actual, cot_completo_segmento, max_input_tokens_segment, max_cot_extract_chars_segment)
         all_segment_inputs.append(input_str)
         segment_identifiers.append({'texto_id': texto_id, 'segmento_idx': seg_idx})

# Llamar a la función genérica de embeddings
segment_results_list = get_embeddings_batch(
    inputs=all_segment_inputs,
    identifiers=segment_identifiers,
    model_name=embedding_model_name_api,
    task_type=embedding_task_type_segment,
    batch_size=batch_size_common,
    sleep_time=sleep_time_common,
    item_type="segmento"
)

# Crear DataFrame base de segmentos
df_segment_embeddings_base = pd.DataFrame(segment_results_list) if segment_results_list else pd.DataFrame()

# ======================================================================
# --- PASO 2: Fusionar Datos JSON en DataFrame de SEGMENTOS ---
# ======================================================================
print("\n\n--- PASO 2: Fusionando Datos JSON en DataFrame de Segmentos ---")
if df_segment_embeddings_base.empty:
    print("(!) No hay DataFrame base de segmentos para fusionar.")
    # Crear DF vacío con columnas esperadas si no existe
    resultados_embeddings_segmentos = pd.DataFrame(columns=['texto_id', 'segmento_idx', 'embedding', 'error', 'analisis_json_completo', 'json_error'])
else:
    json_data_for_segments = []
    if 'resultados_json_final_revisado' in globals():
        for texto_id, data_texto in resultados_json_final_revisado.items():
            for seg_idx, segmento_info in enumerate(data_texto.get("json_por_segmento", [])):
                json_data_for_segments.append({
                    'texto_id': texto_id,
                    'segmento_idx': seg_idx,
                    'analisis_json_completo': segmento_info.get("analisis_json_object"), # La lista de dicts
                    'json_error': segmento_info.get("error") # Error de parseo JSON
                })
        if json_data_for_segments:
            df_json_segment_data = pd.DataFrame(json_data_for_segments)
            print(f"  -> Datos JSON extraídos para {len(df_json_segment_data)} segmentos.")
            # Realizar el merge
            df_segmentos_con_json_y_emb = pd.merge(
                df_segment_embeddings_base,
                df_json_segment_data,
                on=['texto_id', 'segmento_idx'],
                how='left'
            )
            print("  -> Merge realizado.")
            print(f"  Columnas DESPUÉS del merge: {df_segmentos_con_json_y_emb.columns.tolist()}")
            # Asignar a la variable final
            resultados_embeddings_segmentos = df_segmentos_con_json_y_emb
        else:
            print("  (!) No se extrajo información JSON para fusionar.")
            resultados_embeddings_segmentos = df_segment_embeddings_base # Usar el base
            resultados_embeddings_segmentos['analisis_json_completo'] = None # Añadir columna vacía
            resultados_embeddings_segmentos['json_error'] = 'JSON no extraído'
    else:
        print("(!) 'resultados_json_final_revisado' no encontrado. No se puede fusionar JSON.")
        resultados_embeddings_segmentos = df_segment_embeddings_base
        if 'analisis_json_completo' not in resultados_embeddings_segmentos.columns: resultados_embeddings_segmentos['analisis_json_completo'] = None
        if 'json_error' not in resultados_embeddings_segmentos.columns: resultados_embeddings_segmentos['json_error'] = 'JSON no disponible'

# ======================================================================
# --- PASO 3: Generar Embeddings y DataFrame Detallado para PALABRAS ---
# ======================================================================
print("\n\n--- PASO 3: Generando Embeddings y DataFrame Detallado para PALABRAS ---")
all_word_inputs = []; word_identifiers = []; json_details_lookup = {}
palabras_procesadas_count = 0; segmentos_con_error_json_palabras = 0

if 'resultados_json_final_revisado' not in globals():
     print("(!) 'resultados_json_final_revisado' no encontrado. Saltando palabras.")
     resultados_embeddings_palabras = pd.DataFrame() # Crear vacío
else:
    print("Preparando inputs y lookup JSON para palabras...")
    for texto_id, data_texto in resultados_json_final_revisado.items():
        for seg_idx, segmento_info in enumerate(data_texto.get("json_por_segmento", [])):
            palabras_list = segmento_info.get("analisis_json_object")
            if isinstance(palabras_list, list):
                for palabra_idx, palabra_data in enumerate(palabras_list):
                     if isinstance(palabra_data, dict):
                          input_str = prepare_word_input(palabra_data, max_input_tokens_word)
                          all_word_inputs.append(input_str)
                          palabra_real = palabra_data.get('palabra_analizada', palabra_data.get('texto', '?'))
                          word_identifiers.append({'texto_id': texto_id, 'segmento_idx': seg_idx, 'palabra_idx_in_segment': palabra_idx, 'palabra_texto': palabra_real})
                          lookup_key = (texto_id, seg_idx, palabra_idx); json_details_lookup[lookup_key] = palabra_data
                          palabras_procesadas_count += 1
            elif segmento_info.get("error"): segmentos_con_error_json_palabras += 1

    if segmentos_con_error_json_palabras > 0: print(f"(!) Se omitieron palabras de {segmentos_con_error_json_palabras} segmentos por error JSON.")

    # Llamar a la función genérica de embeddings
    word_results_list = get_embeddings_batch(
        inputs=all_word_inputs,
        identifiers=word_identifiers, # Los identificadores base
        model_name=embedding_model_name_api,
        task_type=embedding_task_type_word,
        batch_size=batch_size_common,
        sleep_time=sleep_time_common,
        item_type="palabra"
    )

    # Construir DataFrame Detallado
    print("\nConstruyendo DataFrame final detallado de palabras...")
    word_results_list_detailed = []; num_json_lookup_fallidos = 0
    for result_base in word_results_list: # Iterar sobre los resultados del embedding
        lookup_key = (result_base['texto_id'], result_base['segmento_idx'], result_base['palabra_idx_in_segment'])
        detalles_json = json_details_lookup.get(lookup_key)
        if detalles_json is None:
            num_json_lookup_fallidos += 1; record_completo = {**result_base, 'categoria': 'desconocida_sin_json'}
        else:
            record_completo = {**result_base, **detalles_json} # Fusionar base + detalles
            # Asegurar consistencia
            if 'palabra_analizada' in record_completo and 'palabra_texto' not in result_base: record_completo['palabra_texto'] = record_completo.pop('palabra_analizada')
            if 'categoria' not in record_completo: record_completo['categoria'] = 'desconocida'

        word_results_list_detailed.append(record_completo)

    if word_results_list_detailed:
        df_word_embeddings_detailed = pd.DataFrame(word_results_list_detailed)
        if 'categoria' in df_word_embeddings_detailed.columns: df_word_embeddings_detailed['categoria'] = df_word_embeddings_detailed['categoria'].fillna('desconocida')
        print(f"-> DataFrame DETALLADO de Palabras creado ({len(df_word_embeddings_detailed)} filas).")
        print(f"   Columnas: {df_word_embeddings_detailed.columns.tolist()}")
        resultados_embeddings_palabras = df_word_embeddings_detailed
        if num_json_lookup_fallidos > 0: print(f"(!) {num_json_lookup_fallidos} palabras sin detalles JSON.")
    else:
        print("(!) No se pudieron generar registros detallados para palabras.")
        resultados_embeddings_palabras = pd.DataFrame()

# --- Mostrar Resúmenes ---
print("\n\n--- Resumen de DataFrames Generados ---")
if 'resultados_embeddings_segmentos' in globals() and not resultados_embeddings_segmentos.empty:
    print("\nDataFrame SEGMENTOS (con JSON):")
    display(resultados_embeddings_segmentos.head(2))
    # Verificar si la columna JSON está presente
    if 'analisis_json_completo' in resultados_embeddings_segmentos.columns:
        print("  (Columna 'analisis_json_completo' encontrada ✅)")
        print(f"  Valores no nulos: {resultados_embeddings_segmentos['analisis_json_completo'].notna().sum()}")
    else: print("  (!) Columna 'analisis_json_completo' NO encontrada ❌")
else: print("\n(!) DataFrame de Segmentos no generado.")

if 'resultados_embeddings_palabras' in globals() and not resultados_embeddings_palabras.empty:
    print("\nDataFrame PALABRAS (Detallado):")
    display(resultados_embeddings_palabras.head(2))
else: print("\n(!) DataFrame de Palabras no generado.")


print("\n--- Fin de Celda 11 (Revisado - Unificado) ---")

--- Celda 11 (Revisado): Embeddings Palabras/Segmentos y Fusión ---


--- PASO 1: Generando Embeddings para SEGMENTOS ---
Preparando inputs para embeddings de segmentos...

Iniciando generación para 98 segmentos (Modelo: models/text-embedding-004, Task: RETRIEVAL_DOCUMENT)...
  Procesando Lote Segmento 1/1...
-> Generación segmentos completada. 98 embeddings OK, 0 errores API.


--- PASO 2: Fusionando Datos JSON en DataFrame de Segmentos ---
  -> Datos JSON extraídos para 98 segmentos.
  -> Merge realizado.
  Columnas DESPUÉS del merge: ['texto_id', 'segmento_idx', 'embedding', 'error', 'analisis_json_completo', 'json_error']


--- PASO 3: Generando Embeddings y DataFrame Detallado para PALABRAS ---
Preparando inputs y lookup JSON para palabras...

Iniciando generación para 990 palabras (Modelo: models/text-embedding-004, Task: SEMANTIC_SIMILARITY)...
  Procesando Lote Palabra 1/10...
  Procesando Lote Palabra 2/10...
  Procesando Lote Palabra 3/10...
  Procesando Lote Palabra 4/10...


,texto_id,segmento_idx,embedding,error,analisis_json_completo,json_error
0,Texto_1,0,"[-0.060000945, 0.056589235, -0.02383715, -0.01...",None,"[{'categoria': 'pronombre', 'palabra_analizada...",None
1,Texto_2,0,"[0.007489225, 0.026745263, 0.008314279, 0.0113...",None,"[{'categoria': 'verbo', 'palabra_analizada': '...",None


  (Columna 'analisis_json_completo' encontrada ✅)
  Valores no nulos: 98

DataFrame PALABRAS (Detallado):


,texto_id,segmento_idx,palabra_idx_in_segment,palabra_texto,embedding,error,categoria,palabra_analizada,tipo,persona,...,lemma,diminutivo_comun,aumentativo_comun,definicion_contextual,funcion,grado,modifica_a,apocope,usos_comunes,emocion_tipica
0,Texto_1,0,0,Me,"[-0.0020452668, 0.010883356, -0.021525761, 0.0...",None,pronombre,Me,personal,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Texto_1,0,1,gustaría,"[-0.083154745, 0.035446163, -0.015055345, 0.03...",None,verbo,gustaría,NaN,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN



--- Fin de Celda 11 (Revisado - Unificado) ---


In [116]:
# @title 11.B: Usar CoT Global COMPLETO para Embeddings

import google.generativeai as genai
import time
import numpy as np
import pandas as pd
import re
from google.api_core import exceptions
import textwrap

# --- Verificaciones Previas ---
# (Igual que antes)
if 'model' not in globals() or model is None: raise ValueError("Modelo Gemini no inicializado (Celda 2).")
if 'resultados_cot_global' not in globals() or not resultados_cot_global:
    raise NameError("Resultados CoT Global ('resultados_cot_global') no encontrados (Celda 9).")

# --- Configuración ---
# (Igual que antes)
embedding_model_id_text = "text-embedding-004" # @param ["text-embedding-004", "embedding-001"] {allow-input: true}
embedding_model_name_text_api = f"models/{embedding_model_id_text}"
embedding_task_type_text = "RETRIEVAL_DOCUMENT" #@param ["RETRIEVAL_DOCUMENT", "CLUSTERING", "SEMANTIC_SIMILARITY"]
max_input_tokens_text = 2048
batch_size_text = 50
sleep_time_embedding_text = 15

print(f"Usando modelo embedding para textos: {embedding_model_name_text_api}")
print(f"Task Type para textos: {embedding_task_type_text}")
print("🚨 ADVERTENCIA: Se intentará usar el CoT Global completo. ¡Alto riesgo de exceder límites y truncar!")

# --- Función Auxiliar Modificada ---
def prepare_text_input_FULL_COT(text_original: str, cot_global: str, max_len_tokens: int) -> str:
    """Prepara input usando el CoT Global COMPLETO, truncando si es necesario."""
    if not cot_global or not isinstance(cot_global, str):
        cot_global = "(CoT Global no disponible)" # Usar placeholder si falta

    # Texto base (sin truncar inicialmente)
    base_input = f"Texto Completo:\n{text_original}\n\nAnálisis Global CoT Completo:\n{cot_global}"

    # Estimación tokens (muy básica: chars / 4)
    estimated_tokens = len(base_input) / 4

    if estimated_tokens <= max_len_tokens:
        return base_input # Cabe todo, perfecto
    else:
        # --- Estrategia de Truncamiento ---
        print(f"  🔥 Input ({estimated_tokens:.0f} tokens) excede límite ({max_len_tokens}). TRUNCANDO AGRESIVAMENTE...")
        allowed_chars = int(max_len_tokens * 3.5) # Dejar margen
        overhead = len("Texto Completo:\n\n\nAnálisis Global CoT Completo:\n")

        # Prioridad: Mantener tanto CoT como sea posible.
        # Calcular cuánto espacio queda para el texto original después del overhead y el CoT completo.
        chars_for_original = allowed_chars - overhead - len(cot_global)

        if chars_for_original > 100: # Si queda espacio razonable para el texto original
             truncated_original = text_original[:chars_for_original] + "..."
             truncated_cot = cot_global # Mantener CoT completo
             print(f"     -> Texto original truncado a {len(truncated_original)} chars.")
        else:
             # Si no queda casi espacio para el texto original, truncar AMBOS.
             # Dar ~20% al texto original, 80% al CoT (aproximado)
             chars_for_original = int((allowed_chars - overhead) * 0.2)
             chars_for_cot = (allowed_chars - overhead) - chars_for_original

             truncated_original = text_original[:chars_for_original] + "..."
             truncated_cot = cot_global[:chars_for_cot] + "..."
             print(f"     -> ¡Truncando AMBOS! Texto original a {len(truncated_original)} chars, CoT a {len(truncated_cot)} chars.")

        return f"Texto Completo:\n{truncated_original}\n\nAnálisis Global CoT Completo:\n{truncated_cot}"
        # ---------------------------------


# --- Generar Embeddings para Textos Completos ---
all_text_inputs = []; text_identifiers = []
print("\nPreparando inputs (usando CoT completo) para embeddings...")

for texto_id, data_global in resultados_cot_global.items():
    texto_original = data_global.get("texto_original"); cot_global_final = data_global.get("cot_global_final")
    if texto_original:
         # Llamar a la nueva función de preparación
         input_str = prepare_text_input_FULL_COT(texto_original, cot_global_final, max_input_tokens_text) # <--- CAMBIO AQUÍ
         all_text_inputs.append(input_str); text_identifiers.append({'texto_id': texto_id})
    else: print(f"  Skipping {texto_id}: Texto original no encontrado.")

if not all_text_inputs:
    print("No se prepararon inputs válidos."); df_text_embeddings = pd.DataFrame()
else:
    print(f"Se prepararon {len(all_text_inputs)} inputs."); print(f"Iniciando generación en lotes de {batch_size_text}...")
    text_results_list = []
    total_batches_text = (len(all_text_inputs) + batch_size_text - 1) // batch_size_text
    for i in range(0, len(all_text_inputs), batch_size_text):
        batch_inputs = all_text_inputs[i : i + batch_size_text]; batch_ids = text_identifiers[i : i + batch_size_text]
        batch_num = (i // batch_size_text) + 1
        print(f"  Procesando Lote Textos {batch_num}/{total_batches_text}...")
        try:
            response = genai.embed_content(model=embedding_model_name_text_api, content=batch_inputs, task_type=embedding_task_type_text)
            if 'embedding' in response and isinstance(response['embedding'], list) and len(response['embedding']) == len(batch_inputs):
                print(f"    ✅ Lote {batch_num} OK.")
                for j, identifier in enumerate(batch_ids): text_results_list.append({**identifier, 'embedding': np.array(response['embedding'][j]), 'error': None})
            else:
                 print(f"    ⛔ Error Lote {batch_num}: Respuesta API inesperada."); print(f"      {str(response)[:200]}")
                 for identifier in batch_ids: text_results_list.append({**identifier, 'embedding': None, 'error': 'Respuesta API inesperada'})
        except exceptions.ResourceExhausted as e_rate:
            print(f"    ⚠️ Rate Limit Lote {batch_num}. Esperando {sleep_time_embedding_text}s..."); time.sleep(sleep_time_embedding_text)
            print(f"    ⛔ Lote {batch_num} falló por Rate Limit.");
            for identifier in batch_ids: text_results_list.append({**identifier, 'embedding': None, 'error': f'RateLimitError: {e_rate}'})
        except Exception as e:
            print(f"    ⛔ Error procesando Lote {batch_num}: {e}")
            for identifier in batch_ids: text_results_list.append({**identifier, 'embedding': None, 'error': str(e)})
        if batch_num < total_batches_text: time.sleep(max(1, sleep_time_embedding_text / 3))

    print("\nGeneración de embeddings completada.")
    df_text_embeddings = pd.DataFrame(text_results_list)

if not df_text_embeddings.empty:
    success_count = df_text_embeddings['embedding'].notna().sum(); error_count = df_text_embeddings['error'].notna().sum()
    print(f"\nEmbeddings de Textos: {success_count} éxitos, {error_count} errores.")
    print("\nPrimeras filas:"); display(df_text_embeddings.head())
    df_text_embeddings.info()
else: print("\nNo se generó DataFrame.")

resultados_embeddings_textos = df_text_embeddings

print("\n--- Fin de Celda 11.B (CoT Completo) ---")

Usando modelo embedding para textos: models/text-embedding-004
Task Type para textos: RETRIEVAL_DOCUMENT
🚨 ADVERTENCIA: Se intentará usar el CoT Global completo. ¡Alto riesgo de exceder límites y truncar!

Preparando inputs (usando CoT completo) para embeddings...
Se prepararon 19 inputs.
Iniciando generación en lotes de 50...
  Procesando Lote Textos 1/1...
    ✅ Lote 1 OK.

Generación de embeddings completada.

Embeddings de Textos: 19 éxitos, 0 errores.

Primeras filas:


,texto_id,embedding,error
0,Texto_1,"[-0.037022393, 0.057320707, -0.027191967, 0.01...",None
1,Texto_2,"[0.011024177, 0.038843084, -0.0059259715, 0.01...",None
2,Texto_3,"[0.0021676808, 0.032636456, 0.0039843484, 0.03...",None
3,Texto_4,"[-0.020371169, 0.047818948, -0.06864625, 0.025...",None
4,Texto_5,"[0.0021858974, -0.006759613, -0.04616283, 0.03...",None


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19 entries, 0 to 18
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   texto_id   19 non-null     object
 1   embedding  19 non-null     object
 2   error      0 non-null      object
dtypes: object(3)
memory usage: 588.0+ bytes

--- Fin de Celda 11.B (CoT Completo) ---


In [117]:
print("Columnas en resultados_embeddings_segmentos ANTES de Celda 12:")
if 'resultados_embeddings_segmentos' in globals():
    print(resultados_embeddings_segmentos.columns.tolist())
else:
    print("Variable no encontrada.")

Columnas en resultados_embeddings_segmentos ANTES de Celda 12:
['texto_id', 'segmento_idx', 'embedding', 'error', 'analisis_json_completo', 'json_error']


In [118]:
# @title 12 (Revisado v4): Reducción para TODOS los Niveles (Indentación Corregida)

import time
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import warnings
from IPython.display import display

# --- Importar UMAP ---
try:
    # Intenta importar la implementación moderna de umap
    import umap.umap_ as umap
    umap_available = True
    print("UMAP (umap-learn) encontrado.")
except ImportError:
    try:
        # Intenta importar la implementación antigua (por si acaso)
        import umap
        umap_available = True
        print("UMAP (versión antigua) encontrado.")
    except ImportError:
        umap_available = False
        print("(!) UMAP no encontrado. La reducción con UMAP no estará disponible.")
        print("   Puedes instalarlo con: !pip install umap-learn")

# --- Verificaciones Previas ---
print("\n--- Verificando DataFrames de Embeddings ---")
dfs_to_process = {}
# Verificar y añadir cada DataFrame a procesar
if 'resultados_embeddings_palabras' in globals() and isinstance(resultados_embeddings_palabras, pd.DataFrame) and not resultados_embeddings_palabras.empty:
    dfs_to_process['palabras'] = resultados_embeddings_palabras
    print(f"DataFrame de Palabras encontrado ({len(resultados_embeddings_palabras)} filas).")
else: print("(!) DataFrame de Palabras NO encontrado o vacío.")
if 'resultados_embeddings_segmentos' in globals() and isinstance(resultados_embeddings_segmentos, pd.DataFrame) and not resultados_embeddings_segmentos.empty:
    dfs_to_process['segmentos'] = resultados_embeddings_segmentos
    print(f"DataFrame de Segmentos encontrado ({len(resultados_embeddings_segmentos)} filas).")
else: print("(!) DataFrame de Segmentos NO encontrado o vacío.")
if 'resultados_embeddings_textos' in globals() and isinstance(resultados_embeddings_textos, pd.DataFrame) and not resultados_embeddings_textos.empty:
    dfs_to_process['textos'] = resultados_embeddings_textos
    print(f"DataFrame de Textos encontrado ({len(resultados_embeddings_textos)} filas).")
else: print("(!) DataFrame de Textos NO encontrado o vacío.")

# --- Parámetros de Reducción (Leídos de globals o con defaults) ---
print("\n--- Configuración de Reducción ---")
# Leer parámetros (asegurando valores por defecto si no existen globalmente)
tsne_perplexity = float(globals().get('tsne_perplexity_reduc', 30.0))
tsne_n_iter = int(globals().get('tsne_n_iter_reduc', 1000)) # max_iter
tsne_lr_str = str(globals().get('tsne_learning_rate_reduc', 'auto'))
umap_n_neighbors = int(globals().get('umap_n_neighbors_reduc', 15))
umap_min_dist = float(globals().get('umap_min_dist_reduc', 0.1))
umap_metric = str(globals().get('umap_metric_reduc', 'cosine'))
random_state_seed = int(globals().get('random_state_seed_reduc', 42))
lr_tsne = 'auto' if tsne_lr_str.lower() == 'auto' else float(tsne_lr_str)
rng_state = None if random_state_seed == 0 else random_state_seed
print(f"Params: Perp={tsne_perplexity}, MaxIter={tsne_n_iter}, LR={lr_tsne}, Neigh={umap_n_neighbors}, Dist={umap_min_dist}, Met={umap_metric}, Seed={rng_state}")


# --- Función para aplicar Reducción (Indentación Corregida + Try/Except Internos) ---
def apply_dimensionality_reduction(df_input, name):
    """Aplica PCA, t-SNE, UMAP a un DataFrame de embeddings."""
    print(f"\n--- Procesando Reducción para: {name.upper()} ---")
    # Verificar columna de embedding
    embedding_col = None
    if 'embedding' in df_input.columns: embedding_col = 'embedding'
    elif 'embedding_vector' in df_input.columns: embedding_col = 'embedding_vector'

    if not embedding_col: print(f"  (!) Columna 'embedding' o 'embedding_vector' no encontrada. Saltando."); return df_input

    df_reduc = df_input.copy()
    # Filtrar NaNs en la columna de embedding y asegurar que sean iterables con longitud > 0
    valid_mask = df_reduc[embedding_col].apply(lambda x: x is not None and hasattr(x, '__len__') and len(x) > 0)
    df_valid = df_reduc[valid_mask].copy()
    df_invalid = df_reduc[~valid_mask].copy()

    if df_valid.empty: print("  (!) No hay embeddings válidos para procesar."); return df_reduc

    print(f"  Procesando {len(df_valid)} embeddings válidos...");
    # --- INICIO DEL TRY PRINCIPAL ---
    try:
        # Preparar datos para sklearn/umap
        X = np.array(df_valid[embedding_col].tolist()) # Convertir lista de arrays/listas a array 2D
        print(f"  Dimensiones array: {X.shape}")
        results_2d, results_3d = {}, {}; n_samples = X.shape[0]

        # PCA
        try:
            n_comp_pca = min(3, n_samples)
            if n_comp_pca >= 2:
                pca_2d = PCA(n_components=2, random_state=rng_state); results_2d['pca'] = pca_2d.fit_transform(X); print("  -> PCA 2D OK.")
            if n_comp_pca >= 3:
                pca_3d = PCA(n_components=3, random_state=rng_state); results_3d['pca'] = pca_3d.fit_transform(X); print("  -> PCA 3D OK.")
        except Exception as e_pca: print(f"  -> Error PCA: {e_pca}")

        # t-SNE
        eff_perp = min(tsne_perplexity, max(1.0, n_samples - 1.1))
        tsne_params = {'perplexity': eff_perp, 'max_iter': tsne_n_iter, 'learning_rate': lr_tsne,
                       'init': 'pca', 'random_state': rng_state, 'n_jobs': -1}
        try:
            with warnings.catch_warnings(): warnings.simplefilter("ignore", category=FutureWarning)
            if n_samples > 1:
                 tsne_2d = TSNE(n_components=2, **tsne_params); results_2d['tsne'] = tsne_2d.fit_transform(X); print("  -> t-SNE 2D OK.")
                 tsne_3d = TSNE(n_components=3, **tsne_params); results_3d['tsne'] = tsne_3d.fit_transform(X); print("  -> t-SNE 3D OK.")
            else: print("  -> t-SNE saltado (n_samples <= 1).")
        except Exception as e_tsne: print(f"  -> Error t-SNE: {e_tsne}")

        # UMAP
        if umap_available:
            eff_neigh = min(umap_n_neighbors, n_samples - 1)
            if eff_neigh >= 2:
                 umap_params = {'n_neighbors': eff_neigh, 'min_dist': umap_min_dist, 'metric': umap_metric,
                                'random_state': rng_state, 'n_components': 2, 'n_jobs': -1}
                 try:
                     with warnings.catch_warnings(): warnings.simplefilter("ignore")
                     # Usar ensure_all_finite=False explícitamente si la advertencia persiste y los datos son confiables
                     reducer_2d = umap.UMAP(**umap_params); results_2d['umap'] = reducer_2d.fit_transform(X); print("  -> UMAP 2D OK.")
                     umap_params['n_components'] = 3
                     reducer_3d = umap.UMAP(**umap_params); results_3d['umap'] = reducer_3d.fit_transform(X); print("  -> UMAP 3D OK.")
                 except Exception as e_umap: print(f"  -> Error UMAP: {e_umap}")
            else: print("  -> UMAP saltado (n_neighbors < 2).")

        # Añadir coordenadas a df_valid
        for method, data_2d in results_2d.items(): m_clean = method.replace('-', ''); df_valid[f'{m_clean}_2d_x'] = data_2d[:, 0]; df_valid[f'{m_clean}_2d_y'] = data_2d[:, 1]
        for method, data_3d in results_3d.items(): m_clean = method.replace('-', ''); df_valid[f'{m_clean}_3d_x'] = data_3d[:, 0]; df_valid[f'{m_clean}_3d_y'] = data_3d[:, 1]; df_valid[f'{m_clean}_3d_z'] = data_3d[:, 2]

        # Reunir con inválidos
        new_cols = df_valid.columns.difference(df_input.columns)
        for col in new_cols:
             if col not in df_invalid.columns: df_invalid[col] = np.nan # Asegura que la columna existe en df_invalid
        # Corregir concat para evitar warning de dtype
        if not df_invalid.empty:
             df_final = pd.concat([df_valid, df_invalid]).reindex(df_input.index)
        else: # Si no había inválidos, df_valid ya es el final
             df_final = df_valid
        print(f"  -> Coordenadas (si se calcularon) añadidas para {name}.")
        return df_final

    # --- FIN DEL TRY PRINCIPAL ---
    # --- EXCEPT INDENTADOS CORRECTAMENTE ---
    except ValueError as ve:
         print(f"  ⛔ ERROR al preparar embeddings (np.vstack?) para {name}: {ve}")
         return df_input
    except Exception as e_outer:
         print(f"  ⛔ ERROR inesperado durante reducción para {name}: {e_outer}")
         return df_input
# --- FIN DE LA FUNCIÓN ---


# --- Aplicar Reducción y Guardar Resultados ---
# Definir variables globales ANTES del bucle por si falla algún DF
resultados_palabras_reducidas = None
resultados_segmentos_reducidos = None
resultados_textos_reducidos = None

start_time_all_reduc = time.time()

# Llamar a la función para cada DataFrame disponible
if 'palabras' in dfs_to_process:
    resultados_palabras_reducidas = apply_dimensionality_reduction(dfs_to_process['palabras'], 'palabras')
if 'segmentos' in dfs_to_process:
    # Asegurarse de pasar el DF correcto (el que potencialmente tiene la columna JSON)
    resultados_segmentos_reducidos = apply_dimensionality_reduction(dfs_to_process['segmentos'], 'segmentos')
if 'textos' in dfs_to_process:
    resultados_textos_reducidos = apply_dimensionality_reduction(dfs_to_process['textos'], 'textos')

end_time_all_reduc = time.time(); print(f"\n--- Tiempo total: {end_time_all_reduc - start_time_all_reduc:.2f} segundos ---")


# --- Mostrar Resúmenes (Añadir verificación de columna JSON) ---
print("\n--- Resumen Final ---")
if resultados_palabras_reducidas is not None:
    print("\nDataFrame PALABRAS Reducidas:")
    display(resultados_palabras_reducidas.head(2))
    resultados_palabras_reducidas.info(verbose=False) # Menos verboso para ahorrar espacio
if resultados_segmentos_reducidos is not None:
    print("\nDataFrame SEGMENTOS Reducidos:")
    if 'analisis_json_completo' in resultados_segmentos_reducidos.columns:
         print("  (Columna 'analisis_json_completo' encontrada ✅)")
         print(f"  Valores no nulos: {resultados_segmentos_reducidos['analisis_json_completo'].notna().sum()}")
    else: print("  (!) Columna 'analisis_json_completo' NO encontrada ❌")
    display(resultados_segmentos_reducidos.head(2))
    resultados_segmentos_reducidos.info(verbose=False)
if resultados_textos_reducidos is not None:
    print("\nDataFrame TEXTOS Reducidos:")
    display(resultados_textos_reducidos.head(2))
    resultados_textos_reducidos.info(verbose=False)

print("\n--- Fin de Celda 12 (Revisado v4 - Indentación Corregida) ---")

UMAP (umap-learn) encontrado.

--- Verificando DataFrames de Embeddings ---
DataFrame de Palabras encontrado (990 filas).
DataFrame de Segmentos encontrado (98 filas).
DataFrame de Textos encontrado (19 filas).

--- Configuración de Reducción ---
Params: Perp=30.0, MaxIter=1000, LR=auto, Neigh=15, Dist=0.1, Met=cosine, Seed=42

--- Procesando Reducción para: PALABRAS ---
  Procesando 990 embeddings válidos...
  Dimensiones array: (990, 768)
  -> PCA 2D OK.
  -> PCA 3D OK.
  -> t-SNE 2D OK.
  -> t-SNE 3D OK.


/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning:

'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.

/usr/local/lib/python3.11/dist-packages/umap/umap_.py:1952: UserWarning:

n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.



  -> UMAP 2D OK.


/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning:

'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.

/usr/local/lib/python3.11/dist-packages/umap/umap_.py:1952: UserWarning:

n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.



  -> UMAP 3D OK.
  -> Coordenadas (si se calcularon) añadidas para palabras.

--- Procesando Reducción para: SEGMENTOS ---
  Procesando 98 embeddings válidos...
  Dimensiones array: (98, 768)
  -> PCA 2D OK.
  -> PCA 3D OK.
  -> t-SNE 2D OK.
  -> t-SNE 3D OK.
  -> UMAP 2D OK.


...

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 990 entries, 0 to 989
Columns: 51 entries, texto_id to umap_3d_z
dtypes: float32(10), float64(5), int64(2), object(34)
memory usage: 355.9+ KB

DataFrame SEGMENTOS Reducidos:
  (Columna 'analisis_json_completo' encontrada ✅)
  Valores no nulos: 98


,texto_id,segmento_idx,embedding,error,analisis_json_completo,json_error,pca_2d_x,pca_2d_y,tsne_2d_x,tsne_2d_y,...,umap_2d_y,pca_3d_x,pca_3d_y,pca_3d_z,tsne_3d_x,tsne_3d_y,tsne_3d_z,umap_3d_x,umap_3d_y,umap_3d_z
0,Texto_1,0,"[-0.060000945, 0.056589235, -0.02383715, -0.01...",None,"[{'categoria': 'pronombre', 'palabra_analizada...",None,0.377948,0.042468,1.862156,8.612226,...,7.858905,0.377948,0.042469,-0.128591,-105.71463,24.998026,83.246643,-3.208090,9.606387,7.989783
1,Texto_2,0,"[0.007489225, 0.026745263, 0.008314279, 0.0113...",None,"[{'categoria': 'verbo', 'palabra_analizada': '...",None,0.415345,-0.057667,2.794295,6.716196,...,7.197558,0.415345,-0.057668,-0.146959,-30.62480,-40.568481,66.931892,-3.279914,9.719511,8.337249


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 98 entries, 0 to 97
Columns: 21 entries, texto_id to umap_3d_z
dtypes: float32(10), float64(5), int64(1), object(5)
memory usage: 12.4+ KB

DataFrame TEXTOS Reducidos:


,texto_id,embedding,error,pca_2d_x,pca_2d_y,tsne_2d_x,tsne_2d_y,umap_2d_x,umap_2d_y,pca_3d_x,pca_3d_y,pca_3d_z,tsne_3d_x,tsne_3d_y,tsne_3d_z,umap_3d_x,umap_3d_y,umap_3d_z
0,Texto_1,"[-0.037022393, 0.057320707, -0.027191967, 0.01...",None,0.167254,-0.113606,-19.534506,23.703562,6.937783,-2.265166,0.167444,-0.117096,0.076082,-38.179100,-109.466148,-137.041489,16.556423,5.897613,18.218697
1,Texto_2,"[0.011024177, 0.038843084, -0.0059259715, 0.01...",None,0.158290,0.069908,-28.155733,-4.681897,5.364787,-2.588842,0.157982,0.064946,-0.124827,34.284042,92.569267,101.740501,17.473976,4.569087,18.108936


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19 entries, 0 to 18
Columns: 18 entries, texto_id to umap_3d_z
dtypes: float32(10), float64(5), object(3)
memory usage: 2.1+ KB

--- Fin de Celda 12 (Revisado v4 - Indentación Corregida) ---


In [119]:
print("\nColumnas en resultados_segmentos_reducidos DESPUÉS de Celda 12:")
if 'resultados_segmentos_reducidos' in globals() and resultados_segmentos_reducidos is not None:
    print(resultados_segmentos_reducidos.columns.tolist())
    # Verificar específicamente si la columna está
    if 'analisis_json_completo' in resultados_segmentos_reducidos.columns:
         print("-> ¡Columna 'analisis_json_completo' PRESENTE!")
    else:
         print("-> (!) ¡Columna 'analisis_json_completo' AUSENTE!")
else:
    print("Variable no encontrada o vacía.")


Columnas en resultados_segmentos_reducidos DESPUÉS de Celda 12:
['texto_id', 'segmento_idx', 'embedding', 'error', 'analisis_json_completo', 'json_error', 'pca_2d_x', 'pca_2d_y', 'tsne_2d_x', 'tsne_2d_y', 'umap_2d_x', 'umap_2d_y', 'pca_3d_x', 'pca_3d_y', 'pca_3d_z', 'tsne_3d_x', 'tsne_3d_y', 'tsne_3d_z', 'umap_3d_x', 'umap_3d_y', 'umap_3d_z']
-> ¡Columna 'analisis_json_completo' PRESENTE!


In [120]:
# @title 13: Exportación Completa de Resultados (DataFrames y Diccionarios)

import pandas as pd
import numpy as np
import os
from google.colab import files
import pickle # Para formato Pickle
import json # Para formato JSON grande

print("--- Celda 13: Exportación Completa de Resultados ---")

# --- Verificar Datos a Exportar ---
print("\n--- Verificando datos a exportar ---")
data_to_export = {}
export_flags = {}

# DataFrames (con coordenadas reducidas)
if 'resultados_palabras_reducidas' in globals() and isinstance(resultados_palabras_reducidas, pd.DataFrame) and not resultados_palabras_reducidas.empty:
    data_to_export['df_palabras'] = resultados_palabras_reducidas; export_flags['df_palabras'] = True
else: print("(!) DataFrame de Palabras no disponible."); export_flags['df_palabras'] = False
if 'resultados_segmentos_reducidos' in globals() and isinstance(resultados_segmentos_reducidos, pd.DataFrame) and not resultados_segmentos_reducidos.empty:
    data_to_export['df_segmentos'] = resultados_segmentos_reducidos; export_flags['df_segmentos'] = True
else: print("(!) DataFrame de Segmentos no disponible."); export_flags['df_segmentos'] = False
if 'resultados_textos_reducidos' in globals() and isinstance(resultados_textos_reducidos, pd.DataFrame) and not resultados_textos_reducidos.empty:
    data_to_export['df_textos'] = resultados_textos_reducidos; export_flags['df_textos'] = True
else: print("(!) DataFrame de Textos no disponible."); export_flags['df_textos'] = False

# Diccionarios (CoTs y JSONs)
if 'resultados_analisis_cot' in globals() and isinstance(resultados_analisis_cot, dict) and resultados_analisis_cot:
    data_to_export['dict_cot_segmentos'] = resultados_analisis_cot; export_flags['dict_cot_segmentos'] = True
else: print("(!) Diccionario CoT por Segmento no disponible."); export_flags['dict_cot_segmentos'] = False
if 'resultados_json_final_revisado' in globals() and isinstance(resultados_json_final_revisado, dict) and resultados_json_final_revisado:
    data_to_export['dict_json_palabras'] = resultados_json_final_revisado; export_flags['dict_json_palabras'] = True
else: print("(!) Diccionario JSON por Palabra no disponible."); export_flags['dict_json_palabras'] = False
if 'resultados_cot_global' in globals() and isinstance(resultados_cot_global, dict) and resultados_cot_global:
    data_to_export['dict_cot_global'] = resultados_cot_global; export_flags['dict_cot_global'] = True
else: print("(!) Diccionario CoT Global no disponible."); export_flags['dict_cot_global'] = False

# --- Nombres de Archivo Base ---
base_filenames = {
    'df_palabras': "palabras_reducidas_export",
    'df_segmentos': "segmentos_reducidos_export",
    'df_textos': "textos_reducidos_export",
    'dict_cot_segmentos': "analisis_cot_detallado_export",
    'dict_json_palabras': "analisis_json_palabras_export",
    'dict_cot_global': "analisis_cot_global_export"
}

# --- Opciones de Formato (Simplificado - PKL y CSV por defecto) ---
export_pickle_flag = True # Siempre exportar a Pickle (recomendado)
export_csv_flag = True # Siempre exportar CSV (sin embeddings) para DataFrames
export_json_dict_flag = True # Exportar diccionarios como JSON único

# --- Bucle de Exportación ---
exported_files_list = []

if not any(export_flags.values()): # Verificar si hay algo para exportar
    print("\n(!) No hay datos válidos disponibles para exportar.")
else:
    for name, data_object in data_to_export.items():
        base_filename = base_filenames.get(name, f"export_{name}")
        print(f"\n--- Exportando Objeto: {name.upper()} ---")

        is_dataframe = isinstance(data_object, pd.DataFrame)

        # 1. Exportar a Pickle (.pkl) - Para TODO
        if export_pickle_flag:
            pickle_filename = f"{base_filename}.pkl"
            try:
                print(f"  Guardando en formato Pickle: {pickle_filename}...")
                with open(pickle_filename, 'wb') as f:
                    pickle.dump(data_object, f)
                print(f"  -> Guardado exitosamente.")
                exported_files_list.append(pickle_filename)
            except Exception as e:
                print(f"  ⛔ Error al guardar en Pickle: {e}")

        # 2. Exportar a CSV (.csv) - SOLO para DataFrames (SIN embedding)
        if is_dataframe and export_csv_flag:
            csv_filename = f"{base_filename}_no_embedding.csv"
            try:
                print(f"  Guardando DataFrame en CSV (SIN embedding): {csv_filename}...")
                df = data_object # Renombrar por claridad
                # Identificar columna de embedding
                embedding_col_name = None
                if 'embedding_vector' in df.columns: embedding_col_name = 'embedding_vector'
                elif 'embedding' in df.columns: embedding_col_name = 'embedding'

                if embedding_col_name:
                     cols_to_export_csv = [col for col in df.columns if col != embedding_col_name]
                else:
                     cols_to_export_csv = df.columns.tolist()

                df[cols_to_export_csv].to_csv(csv_filename, index=False, encoding='utf-8-sig')
                print(f"  -> Guardado exitosamente.")
                exported_files_list.append(csv_filename)
            except Exception as e:
                print(f"  ⛔ Error al guardar en CSV: {e}")

        # 3. Exportar a JSON (.json) - SOLO para Diccionarios
        if not is_dataframe and export_json_dict_flag:
             json_filename = f"{base_filename}.json"
             try:
                 print(f"  Guardando Diccionario en JSON: {json_filename}...")
                 # Función auxiliar para manejar tipos no serializables (como NumPy arrays si estuvieran)
                 def default_serializer(obj):
                     if isinstance(obj, np.ndarray):
                         return obj.tolist() # Convertir arrays a listas
                     # Puedes añadir más conversiones aquí si es necesario
                     try:
                         # Intentar conversión estándar
                         return str(obj)
                     except TypeError:
                         return f"<{type(obj).__name__} object not serializable>"

                 with open(json_filename, 'w', encoding='utf-8') as f:
                     json.dump(data_object, f, ensure_ascii=False, indent=2, default=default_serializer)
                 print(f"  -> Guardado exitosamente.")
                 exported_files_list.append(json_filename)
             except Exception as e:
                 print(f"  ⛔ Error al guardar Diccionario en JSON: {e}")
                 # import traceback; traceback.print_exc() # Descomentar para más detalle

# --- Instrucciones para Descarga ---
print("\n--- Descarga de Archivos ---")
if exported_files_list:
    unique_files = sorted(list(set(exported_files_list)))
    print("Los archivos generados se encuentran en el panel de Archivos a la izquierda.")
    print("Puedes descargarlos haciendo clic derecho sobre ellos y seleccionando 'Descargar'.")
    print("\nArchivos generados:")
    for fname in unique_files:
        if os.path.exists(fname):
             # Estimar tamaño más robustamente
             try: size_kb = os.path.getsize(fname) / 1024
             except OSError: size_kb = -1.0
             print(f"- {fname} ({size_kb:.1f} KB)" if size_kb >= 0 else f"- {fname} (Error tamaño)")
        else:
             print(f"- {fname} (¡Error al verificar existencia!)")
    # Opción de descarga automática (descomentar con precaución)
    # print("\nDescargando automáticamente (puede tardar o fallar con archivos grandes)...")
    # for fname in unique_files:
    #     try:
    #         print(f"  Descargando {fname}...")
    #         files.download(fname)
    #     except Exception as e_download:
    #         print(f"  (!) Error al descargar {fname}: {e_download}")

else:
    print("No se generó ningún archivo para descargar (o hubo errores).")

print("\n--- Fin de Celda 13 (Exportación Completa) ---")

--- Celda 13: Exportación Completa de Resultados ---

--- Verificando datos a exportar ---

--- Exportando Objeto: DF_PALABRAS ---
  Guardando en formato Pickle: palabras_reducidas_export.pkl...
  -> Guardado exitosamente.
  Guardando DataFrame en CSV (SIN embedding): palabras_reducidas_export_no_embedding.csv...
  -> Guardado exitosamente.

--- Exportando Objeto: DF_SEGMENTOS ---
  Guardando en formato Pickle: segmentos_reducidos_export.pkl...
  -> Guardado exitosamente.
  Guardando DataFrame en CSV (SIN embedding): segmentos_reducidos_export_no_embedding.csv...
  -> Guardado exitosamente.

--- Exportando Objeto: DF_TEXTOS ---
  Guardando en formato Pickle: textos_reducidos_export.pkl...
  -> Guardado exitosamente.
  Guardando DataFrame en CSV (SIN embedding): textos_reducidos_export_no_embedding.csv...
  -> Guardado exitosamente.

--- Exportando Objeto: DICT_COT_SEGMENTOS ---
  Guardando en formato Pickle: analisis_cot_detallado_export.pkl...
  -> Guardado exitosamente.
  Guardando D

# Estructura de los Datos Exportados

Aquí se describe la estructura de los principales archivos de datos generados por el proceso de análisis lingüístico y de embeddings. Los archivos `.pkl` (Pickle) contienen estas estructuras directamente como objetos Python (DataFrames o diccionarios). Los archivos `.csv` y `.json` representan estos datos en formatos de texto plano.

---

## 1. DataFrames de Coordenadas Reducidas (`*_reducidas_export.*`)

Estos archivos (Pickle y CSV sin embeddings/objetos complejos) contienen información a nivel de palabra, segmento o texto completo, junto con sus representaciones en 2D y 3D obtenidas mediante PCA, t-SNE y UMAP.

**Archivos Base:** `palabras_reducidas_export`, `segmentos_reducidos_export`, `textos_reducidos_export`

**Formato Principal:** Tabla (DataFrame de Pandas en Pickle, CSV)

**A. DataFrame de PALABRAS (`palabras_reducidas_export.*`)**

Contiene información detallada y coordenadas para cada palabra/token analizado.

| Columna                       | Tipo Dato      | Descripción                                                                   | Origen (Celda) | En CSV? |
| :---------------------------- | :------------- | :---------------------------------------------------------------------------- | :------------- | :------ |
| `texto_id`                    | object (str)   | Identificador único del texto original.                                      | Celda 11.A     | Sí      |
| `segmento_idx`                | int64          | Índice del segmento al que pertenece la palabra.                              | Celda 11.A     | Sí      |
| `palabra_idx_in_segment`      | int64          | Índice de la palabra dentro de su segmento JSON.                              | Celda 11.A     | Sí      |
| `palabra_texto`               | object (str)   | El texto literal de la palabra/token.                                          | Celda 11.A     | Sí      |
| `embedding`                   | object (array) | Vector de embedding original de la palabra.                                   | Celda 11.A     | **No**  |
| `error` (embedding)         | object (str)   | Error si falló el embedding para esta palabra.                                | Celda 11.A     | Sí      |
| `categoria`                   | object (str)   | Categoría gramatical principal asignada (sustantivo, verbo, etc.).           | Celda 7/11.A   | Sí      |
| **(Columnas JSON Detalladas)**| **Varios**     | **Todas las demás claves del diccionario JSON** (ej. `lemma`, `genero`, `numero`, `tipo`, `modo`, `tiempo`, `definicion_contextual`, `subtipo`, etc.) se convierten en columnas separadas. El tipo de dato varía. | **Celda 7/11.A (Fusión)** | **Sí**  |
| `pca_2d_x`, `pca_2d_y`        | float64/float32| Coordenadas 2D PCA.                                                           | Celda 12       | Sí      |
| `tsne_2d_x`, `tsne_2d_y`      | float32        | Coordenadas 2D t-SNE.                                                         | Celda 12       | Sí      |
| `umap_2d_x`, `umap_2d_y`      | float32        | Coordenadas 2D UMAP.                                                          | Celda 12       | Sí      |
| `pca_3d_x`, `pca_3d_y`, `pca_3d_z` | float64/float32| Coordenadas 3D PCA.                                                           | Celda 12       | Sí      |
| `tsne_3d_x`, `tsne_3d_y`, `tsne_3d_z`| float32        | Coordenadas 3D t-SNE.                                                         | Celda 12       | Sí      |
| `umap_3d_x`, `umap_3d_y`, `umap_3d_z`| float32        | Coordenadas 3D UMAP.                                                          | Celda 12       | Sí      |

**B. DataFrame de SEGMENTOS (`segmentos_reducidos_export.*`)**

Contiene información y coordenadas para cada segmento, **incluyendo ahora el análisis JSON completo de sus palabras.**

| Columna                       | Tipo Dato         | Descripción                                                                   | Origen (Celda)| En CSV? |
| :---------------------------- | :---------------- | :---------------------------------------------------------------------------- | :------------ | :------ |
| `texto_id`                    | object (str)      | Identificador único del texto original.                                      | Celda 11      | Sí      |
| `segmento_idx`                | int64             | Índice numérico (basado en 0) del segmento.                                  | Celda 11      | Sí      |
| `embedding`                   | object (array)    | Vector de embedding original del segmento.                                   | Celda 11      | **No**  |
| `error` (embedding)         | object (str)      | Error si falló el embedding del segmento.                                     | Celda 11      | Sí      |
| **`analisis_json_completo`**  | **object (list)** | **Lista de diccionarios, cada uno es el JSON detallado de una palabra del segmento.** | **Celda 7/11 (Fusión)** | **No**  |
| **`json_error`**              | **object (str)**  | **Error si falló la generación/parseo del JSON para este segmento.**        | **Celda 7/11 (Fusión)** | Sí      |
| `pca_2d_x`, `pca_2d_y`        | float64/float32   | Coordenadas 2D PCA.                                                           | Celda 12      | Sí      |
| `tsne_2d_x`, `tsne_2d_y`      | float32           | Coordenadas 2D t-SNE.                                                         | Celda 12      | Sí      |
| `umap_2d_x`, `umap_2d_y`      | float32           | Coordenadas 2D UMAP.                                                          | Celda 12      | Sí      |
| `pca_3d_x`, `pca_3d_y`, `pca_3d_z` | float64/float32   | Coordenadas 3D PCA.                                                           | Celda 12      | Sí      |
| `tsne_3d_x`, `tsne_3d_y`, `tsne_3d_z`| float32           | Coordenadas 3D t-SNE.                                                         | Celda 12      | Sí      |
| `umap_3d_x`, `umap_3d_y`, `umap_3d_z`| float32           | Coordenadas 3D UMAP.                                                          | Celda 12      | Sí      |
| `segmento_texto_lookup` (o `segmento_texto`) | object (str) | Texto literal del segmento (nombre puede variar según implementación). | Celda 6/7/Viz | Sí      |
| `hover_text`                  | object (str)      | Texto formateado para tooltips (puede variar según implementación).          | Celda Viz     | Sí      |

**C. DataFrame de TEXTOS COMPLETOS (`textos_reducidos_export.*`)**

Contiene información y coordenadas para cada texto completo.

| Columna                       | Tipo Dato      | Descripción                                                                   | Origen (Celda)| En CSV? |
| :---------------------------- | :------------- | :---------------------------------------------------------------------------- | :------------ | :------ |
| `texto_id`                    | object (str)   | Identificador único del texto original.                                      | Celda 11.B    | Sí      |
| `embedding`                   | object (array) | Vector de embedding original del texto completo.                              | Celda 11.B    | **No**  |
| `error` (embedding)         | object (str)   | Error si falló el embedding del texto.                                        | Celda 11.B    | Sí      |
| `pca_2d_x`, `pca_2d_y`        | float64/float32| Coordenadas 2D PCA.                                                           | Celda 12      | Sí      |
| ... (resto de columnas de coordenadas 2D/3D) ... | ...            | ...                                                                           | Celda 12      | Sí      |
| `umap_3d_x`, `umap_3d_y`, `umap_3d_z`| float32        | Coordenadas 3D UMAP.                                                          | Celda 12      | Sí      |
| `texto_snippet` (Opcional)   | object(str)    | Fragmento del texto original (añadido en cuaderno Viz).                        | Celda Viz     | Sí      |
| `hover_text_text` (Opcional)| object(str)    | Texto formateado para hover (añadido en cuaderno Viz).                         | Celda Viz     | Sí      |

**Nota General sobre CSV:** Los archivos CSV generados (`_no_embedding.csv`) excluyen las columnas `embedding`/`embedding_vector` y `analisis_json_completo` porque contienen objetos complejos (arrays, listas de diccionarios) que no se representan bien en formato tabular simple. Todas las demás columnas (IDs, coordenadas, detalles de palabras aplanados, errores) sí se incluyen.

---

## 2. Diccionario de Análisis CoT Detallado por Segmento (`analisis_cot_detallado_export.*`)

**(Sin cambios)** Esta descripción sigue siendo correcta. Guarda el diccionario `resultados_analisis_cot` (output de Celda 6 original).

---

## 3. Diccionario de Análisis JSON por Palabra (`analisis_json_palabras_export.*`)

**(Sin cambios)** Esta descripción sigue siendo correcta. Guarda el diccionario `resultados_json_final_revisado` (output de Celda 7 original), que contiene la estructura anidada con la lista `analisis_json_object` para cada segmento.

---

## 4. Diccionario de Análisis CoT Global (`analisis_cot_global_export.*`)

**(Sin cambios)** Esta descripción sigue siendo correcta. Guarda el diccionario `resultados_cot_global` (output de Celda 9 original).

---

Esta estructura permite almacenar todos los niveles de análisis, desde los detalles por palabra (ahora como columnas en `palabras_reducidas`) y el JSON completo por segmento (en `segmentos_reducidos`), hasta la síntesis global, junto con las representaciones vectoriales y reducidas, de una manera organizada y completa.
